## Install needed libraries

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

path = "/content/gdrive/MyDrive/Chuyên đề nghiên cứu 1/vnquant_package"
%cd {path}
!ls

In [ ]:
!git clone https://github.com/phamdinhkhanh/vnquant
%cd vnquant
!python setup.py install

In [ ]:
%cd ..
!rm -rf vnquant
!ls

In [4]:
!pip freeze | grep vnquant

vnquant==0.1.1


In [5]:
import vnquant
vnquant.__version__

'0.1.1'

## Import libraries

In [6]:
import vnquant.data as dt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## We choose 10 codes from biggest companies on stock exchange in Vietnam

In [7]:
stock_symbols = ['VIC', 'VCB', 'VNM', 'GAS', 'HVN', 'MWG', 'FPT', 'HPG', 'MSN', 'SAB']

## Prepare and process the dataset

In [8]:
stock = {}
for i in range(len(stock_symbols)):
  data = dt.DataLoader(symbols=stock_symbols[i],
                        start="2014-01-01",
                        end="2024-01-01",
                        data_source="VND")
  stock[stock_symbols[i]] = data.download()

In [9]:
stock[stock_symbols[0]]

Attributes,high,low,open,close,avg,volume
Symbols,VIC,VIC,VIC,VIC,VIC,VIC
date,,,,,,
2014-01-02,70.50,69.50,70.00,70.00,69.92,74950.0
2014-01-03,70.50,69.50,70.00,70.50,70.00,91920.0
2014-01-06,70.50,69.50,70.50,70.50,70.11,111760.0
2014-01-07,70.50,70.00,70.50,70.50,70.47,294120.0
2014-01-08,70.50,70.00,70.00,70.00,70.02,109890.0
...,...,...,...,...,...,...
2023-12-25,43.55,43.00,43.10,43.40,43.34,3364500.0
2023-12-26,43.75,43.35,43.40,43.55,43.55,1806700.0


In [10]:
def reconstruct_data(list_price):
  week = []
  combine = []
  i = 0
  while (i < len(list_price)):
    if (len(week) != 4):
      week.append(np.round(np.mean(list_price[i:i+7]), 2))
      i = i + 7
    else:
      combine.append(week)
      week = []

  return combine

## VIC

In [12]:
close = stock[stock_symbols[0]]['close'].values
open = stock[stock_symbols[0]]['open'].values
high = stock[stock_symbols[0]]['high'].values
low = stock[stock_symbols[0]]['low'].values
avg = stock[stock_symbols[0]]['avg'].values
volume = stock[stock_symbols[0]]['volume'].values

In [13]:
reconstruct_close = reconstruct_data(close)

In [14]:
df_vic = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[0]}'), ('close_week2', f'{stock_symbols[0]}'),
                                  ('close_week3', f'{stock_symbols[0]}'), ('close_week4', f'{stock_symbols[0]}')], dtype = np.dtype(np.float64))
df_vic.head()

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)"
0,70.43,72.93,75.93,77.43
1,77.71,74.64,77.14,75.50
2,72.36,66.79,65.93,63.43
3,66.07,67.21,65.21,64.07
4,64.14,69.21,69.86,72.21


In [16]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [17]:
df_vic[('close_total4week', f'{stock_symbols[0]}')] = mean_close[:len(df_vic)]
df_vic[('open_total4week', f'{stock_symbols[0]}')] = mean_open[:len(df_vic)]
df_vic[('high_total4week', f'{stock_symbols[0]}')] = mean_high[:len(df_vic)]
df_vic[('low_total4week', f'{stock_symbols[0]}')] = mean_low[:len(df_vic)]
df_vic[('avg_total4week', f'{stock_symbols[0]}')] = mean_avg[:len(df_vic)]
df_vic[('volume_total4week', f'{stock_symbols[0]}')] = mean_volume[:len(df_vic)]

In [18]:
df_vic

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)","(close_total4week, VIC)","(open_total4week, VIC)","(high_total4week, VIC)","(low_total4week, VIC)","(avg_total4week, VIC)","(volume_total4week, VIC)"
0,70.43,72.93,75.93,77.43,74.18,74.12,74.82,73.20,74.03,259468.93
1,77.71,74.64,77.14,75.50,76.25,76.62,77.11,75.61,76.32,401583.57
2,72.36,66.79,65.93,63.43,67.12,67.21,68.00,66.20,67.00,669534.64
3,66.07,67.21,65.21,64.07,65.64,65.70,66.29,64.96,65.61,247855.36
4,64.14,69.21,69.86,72.21,68.86,68.93,69.57,68.21,68.87,429015.71
...,...,...,...,...,...,...,...,...,...,...
84,53.06,52.00,50.80,52.00,51.96,52.16,52.52,51.67,52.02,2993792.86
85,52.04,62.04,71.27,64.44,62.45,63.04,63.80,61.65,62.67,13572447.36
86,61.00,55.26,47.25,45.59,52.28,53.05,53.66,51.69,52.57,14991744.82
87,44.86,42.80,41.59,44.69,43.48,43.54,43.98,42.68,43.32,6670760.71


## VCB

In [19]:
close = stock[stock_symbols[1]]['close'].values
open = stock[stock_symbols[1]]['open'].values
high = stock[stock_symbols[1]]['high'].values
low = stock[stock_symbols[1]]['low'].values
avg = stock[stock_symbols[1]]['avg'].values
volume = stock[stock_symbols[1]]['volume'].values

In [20]:
reconstruct_close = reconstruct_data(close)

In [21]:
df_vcb = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[1]}'), ('close_week2', f'{stock_symbols[1]}'),
                                  ('close_week3', f'{stock_symbols[1]}'), ('close_week4', f'{stock_symbols[1]}')], dtype = np.dtype(np.float64))
df_vcb.head()

,"(close_week1, VCB)","(close_week2, VCB)","(close_week3, VCB)","(close_week4, VCB)"
0,27.56,28.26,28.56,28.87
1,29.71,30.20,30.86,31.00
2,31.29,29.70,28.63,25.37
3,26.87,28.60,28.79,27.53
4,26.16,25.97,25.96,25.76


In [22]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [23]:
df_vcb[('close_total4week', f'{stock_symbols[1]}')] = mean_close[:len(df_vcb)]
df_vcb[('open_total4week', f'{stock_symbols[1]}')] = mean_open[:len(df_vcb)]
df_vcb[('high_total4week', f'{stock_symbols[1]}')] = mean_high[:len(df_vcb)]
df_vcb[('low_total4week', f'{stock_symbols[1]}')] = mean_low[:len(df_vcb)]
df_vcb[('avg_total4week', f'{stock_symbols[1]}')] = mean_avg[:len(df_vcb)]
df_vcb[('volume_total4week', f'{stock_symbols[1]}')] = mean_volume[:len(df_vcb)]

In [24]:
df_vcb

,"(close_week1, VCB)","(close_week2, VCB)","(close_week3, VCB)","(close_week4, VCB)","(close_total4week, VCB)","(open_total4week, VCB)","(high_total4week, VCB)","(low_total4week, VCB)","(avg_total4week, VCB)","(volume_total4week, VCB)"
0,27.56,28.26,28.56,28.87,28.31,28.30,28.57,28.02,28.31,1135513.93
1,29.71,30.20,30.86,31.00,30.44,30.42,30.83,30.14,30.48,1103760.71
2,31.29,29.70,28.63,25.37,28.75,28.90,29.25,28.38,28.74,780531.43
3,26.87,28.60,28.79,27.53,27.95,27.82,28.20,27.49,27.90,547587.50
4,26.16,25.97,25.96,25.76,25.96,25.98,26.14,25.81,25.96,301797.14
...,...,...,...,...,...,...,...,...,...,...
84,102.17,100.47,101.91,104.93,102.37,102.20,103.37,101.37,102.37,876050.00
85,96.00,90.73,89.27,87.59,90.90,91.05,91.67,90.31,90.91,1370264.39
86,89.43,88.97,87.99,85.67,88.01,88.05,88.65,87.15,87.94,1487303.57
87,85.93,85.21,88.54,86.67,86.59,86.36,87.12,85.54,86.31,1300789.29


## VNM

In [25]:
close = stock[stock_symbols[2]]['close'].values
open = stock[stock_symbols[2]]['open'].values
high = stock[stock_symbols[2]]['high'].values
low = stock[stock_symbols[2]]['low'].values
avg = stock[stock_symbols[2]]['avg'].values
volume = stock[stock_symbols[2]]['volume'].values

In [26]:
reconstruct_close = reconstruct_data(close)

In [27]:
df_vnm = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[2]}'), ('close_week2', f'{stock_symbols[2]}'),
                                  ('close_week3', f'{stock_symbols[2]}'), ('close_week4', f'{stock_symbols[2]}')], dtype = np.dtype(np.float64))
df_vnm.head()

,"(close_week1, VNM)","(close_week2, VNM)","(close_week3, VNM)","(close_week4, VNM)"
0,136.00,138.71,140.57,139.57
1,139.43,139.57,145.86,141.57
2,144.29,141.00,139.14,127.29
3,123.14,123.43,123.29,121.57
4,122.43,128.14,132.00,135.86


In [28]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [29]:
df_vnm[('close_total4week', f'{stock_symbols[2]}')] = mean_close[:len(df_vnm)]
df_vnm[('open_total4week', f'{stock_symbols[2]}')] = mean_open[:len(df_vnm)]
df_vnm[('high_total4week', f'{stock_symbols[2]}')] = mean_high[:len(df_vnm)]
df_vnm[('low_total4week', f'{stock_symbols[2]}')] = mean_low[:len(df_vnm)]
df_vnm[('avg_total4week', f'{stock_symbols[2]}')] = mean_avg[:len(df_vnm)]
df_vnm[('volume_total4week', f'{stock_symbols[2]}')] = mean_volume[:len(df_vnm)]

In [30]:
df_vnm

,"(close_week1, VNM)","(close_week2, VNM)","(close_week3, VNM)","(close_week4, VNM)","(close_total4week, VNM)","(open_total4week, VNM)","(high_total4week, VNM)","(low_total4week, VNM)","(avg_total4week, VNM)","(volume_total4week, VNM)"
0,136.00,138.71,140.57,139.57,138.71,138.61,139.82,137.79,138.74,173054.64
1,139.43,139.57,145.86,141.57,141.61,141.71,143.18,140.50,141.78,368953.57
2,144.29,141.00,139.14,127.29,137.93,138.64,139.82,136.68,137.99,281483.21
3,123.14,123.43,123.29,121.57,122.86,122.96,124.14,121.96,122.93,130588.57
4,122.43,128.14,132.00,135.86,129.61,129.21,130.46,128.32,129.32,176452.86
...,...,...,...,...,...,...,...,...,...,...
84,66.39,70.09,71.06,72.83,70.09,70.02,70.61,69.46,69.98,5269664.29
85,75.39,74.31,73.26,74.86,74.45,74.16,74.96,73.49,74.28,3663646.43
86,79.24,79.20,76.40,74.14,77.25,77.13,78.01,76.39,77.18,2595932.14
87,74.43,69.20,70.36,69.84,70.96,71.17,71.78,70.23,70.97,2294228.57


## GAS

In [31]:
close = stock[stock_symbols[3]]['close'].values
open = stock[stock_symbols[3]]['open'].values
high = stock[stock_symbols[3]]['high'].values
low = stock[stock_symbols[3]]['low'].values
avg = stock[stock_symbols[3]]['avg'].values
volume = stock[stock_symbols[3]]['volume'].values

In [32]:
reconstruct_close = reconstruct_data(close)

In [33]:
df_gas = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[3]}'), ('close_week2', f'{stock_symbols[3]}'),
                                  ('close_week3', f'{stock_symbols[3]}'), ('close_week4', f'{stock_symbols[3]}')], dtype = np.dtype(np.float64))
df_gas.head()

,"(close_week1, GAS)","(close_week2, GAS)","(close_week3, GAS)","(close_week4, GAS)"
0,67.36,75.07,76.50,81.36
1,83.71,83.00,84.93,83.00
2,87.36,92.14,95.43,93.79
3,90.36,95.36,103.14,105.29
4,109.29,110.57,110.71,112.43


In [34]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [35]:
df_gas[('close_total4week', f'{stock_symbols[3]}')] = mean_close[:len(df_gas)]
df_gas[('open_total4week', f'{stock_symbols[3]}')] = mean_open[:len(df_gas)]
df_gas[('high_total4week', f'{stock_symbols[3]}')] = mean_high[:len(df_gas)]
df_gas[('low_total4week', f'{stock_symbols[3]}')] = mean_low[:len(df_gas)]
df_gas[('avg_total4week', f'{stock_symbols[3]}')] = mean_avg[:len(df_gas)]
df_gas[('volume_total4week', f'{stock_symbols[3]}')] = mean_volume[:len(df_gas)]

In [36]:
df_gas

,"(close_week1, GAS)","(close_week2, GAS)","(close_week3, GAS)","(close_week4, GAS)","(close_total4week, GAS)","(open_total4week, GAS)","(high_total4week, GAS)","(low_total4week, GAS)","(avg_total4week, GAS)","(volume_total4week, GAS)"
0,67.36,75.07,76.50,81.36,75.07,74.93,76.04,74.07,75.06,544727.50
1,83.71,83.00,84.93,83.00,83.66,84.07,84.75,83.04,83.89,325925.36
2,87.36,92.14,95.43,93.79,92.18,92.07,93.43,90.70,91.96,521151.43
3,90.36,95.36,103.14,105.29,98.54,98.04,99.73,96.80,98.28,401566.43
4,109.29,110.57,110.71,112.43,110.75,110.43,111.75,109.21,110.43,277294.64
...,...,...,...,...,...,...,...,...,...,...
84,95.46,95.16,95.61,98.74,96.24,96.38,97.13,95.62,96.35,716707.14
85,99.17,100.59,100.33,101.17,100.31,100.44,101.10,99.70,100.36,538317.86
86,100.79,107.41,92.40,86.11,96.68,96.86,97.79,95.51,96.64,617839.29
87,84.97,79.30,78.51,79.37,80.54,80.82,81.49,79.71,80.47,546039.29


## HVN

In [37]:
close = stock[stock_symbols[4]]['close'].values
open = stock[stock_symbols[4]]['open'].values
high = stock[stock_symbols[4]]['high'].values
low = stock[stock_symbols[4]]['low'].values
avg = stock[stock_symbols[4]]['avg'].values
volume = stock[stock_symbols[4]]['volume'].values

In [38]:
reconstruct_close = reconstruct_data(close)

In [39]:
df_hvn = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[4]}'), ('close_week2', f'{stock_symbols[4]}'),
                                  ('close_week3', f'{stock_symbols[4]}'), ('close_week4', f'{stock_symbols[4]}')], dtype = np.dtype(np.float64))
df_hvn.head()

,"(close_week1, HVN)","(close_week2, HVN)","(close_week3, HVN)","(close_week4, HVN)"
0,43.69,40.24,38.47,41.14
1,38.44,33.83,31.19,30.46
2,28.30,25.79,26.33,26.19
3,28.20,26.83,26.77,26.97
4,26.73,26.66,26.47,26.27


In [40]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [41]:
df_hvn[('close_total4week', f'{stock_symbols[4]}')] = mean_close[:len(df_hvn)]
df_hvn[('open_total4week', f'{stock_symbols[4]}')] = mean_open[:len(df_hvn)]
df_hvn[('high_total4week', f'{stock_symbols[4]}')] = mean_high[:len(df_hvn)]
df_hvn[('low_total4week', f'{stock_symbols[4]}')] = mean_low[:len(df_hvn)]
df_hvn[('avg_total4week', f'{stock_symbols[4]}')] = mean_avg[:len(df_hvn)]
df_hvn[('volume_total4week', f'{stock_symbols[4]}')] = mean_volume[:len(df_hvn)]

In [42]:
df_hvn

,"(close_week1, HVN)","(close_week2, HVN)","(close_week3, HVN)","(close_week4, HVN)","(close_total4week, HVN)","(open_total4week, HVN)","(high_total4week, HVN)","(low_total4week, HVN)","(avg_total4week, HVN)","(volume_total4week, HVN)"
0,43.69,40.24,38.47,41.14,40.89,41.42,41.99,40.08,40.95,184819.00
1,38.44,33.83,31.19,30.46,33.48,33.97,34.66,32.58,33.48,1792022.89
2,28.30,25.79,26.33,26.19,26.65,26.86,27.26,26.30,26.70,593148.07
3,28.20,26.83,26.77,26.97,27.19,27.17,27.57,26.88,27.20,433548.82
4,26.73,26.66,26.47,26.27,26.53,26.49,26.70,26.31,26.53,286891.11
...,...,...,...,...,...,...,...,...,...,...
57,13.46,13.51,14.29,13.20,13.61,13.65,13.85,13.49,13.63,2670992.86
58,12.99,12.97,13.24,13.04,13.06,13.04,13.24,12.82,13.05,2179442.86
59,12.75,13.24,12.67,11.81,12.62,12.68,12.83,12.40,12.64,2129121.43
60,11.64,11.34,10.64,10.79,11.10,11.13,11.29,10.94,11.10,1135360.71


## MWG

In [43]:
close = stock[stock_symbols[5]]['close'].values
open = stock[stock_symbols[5]]['open'].values
high = stock[stock_symbols[5]]['high'].values
low = stock[stock_symbols[5]]['low'].values
avg = stock[stock_symbols[5]]['avg'].values
volume = stock[stock_symbols[5]]['volume'].values

In [44]:
reconstruct_close = reconstruct_data(close)

In [45]:
df_mwg = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[5]}'), ('close_week2', f'{stock_symbols[5]}'),
                                  ('close_week3', f'{stock_symbols[5]}'), ('close_week4', f'{stock_symbols[5]}')], dtype = np.dtype(np.float64))
df_mwg.head()

,"(close_week1, MWG)","(close_week2, MWG)","(close_week3, MWG)","(close_week4, MWG)"
0,97.43,106.86,106.43,134.57
1,137.57,143.00,157.14,168.86
2,148.71,109.86,106.71,102.00
3,100.57,93.71,95.71,99.57
4,104.86,107.86,116.86,120.43


In [46]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [47]:
df_mwg[('close_total4week', f'{stock_symbols[5]}')] = mean_close[:len(df_mwg)]
df_mwg[('open_total4week', f'{stock_symbols[5]}')] = mean_open[:len(df_mwg)]
df_mwg[('high_total4week', f'{stock_symbols[5]}')] = mean_high[:len(df_mwg)]
df_mwg[('low_total4week', f'{stock_symbols[5]}')] = mean_low[:len(df_mwg)]
df_mwg[('avg_total4week', f'{stock_symbols[5]}')] = mean_avg[:len(df_mwg)]
df_mwg[('volume_total4week', f'{stock_symbols[5]}')] = mean_volume[:len(df_mwg)]

In [48]:
df_mwg

,"(close_week1, MWG)","(close_week2, MWG)","(close_week3, MWG)","(close_week4, MWG)","(close_total4week, MWG)","(open_total4week, MWG)","(high_total4week, MWG)","(low_total4week, MWG)","(avg_total4week, MWG)","(volume_total4week, MWG)"
0,97.43,106.86,106.43,134.57,111.32,110.73,112.39,109.16,110.83,83082.50
1,137.57,143.00,157.14,168.86,151.64,152.32,154.07,148.86,151.47,34384.29
2,148.71,109.86,106.71,102.00,116.82,117.21,118.54,115.36,116.77,59553.93
3,100.57,93.71,95.71,99.57,97.39,97.54,98.54,96.23,97.40,74674.29
4,104.86,107.86,116.86,120.43,112.50,111.46,113.32,110.57,112.10,102131.43
...,...,...,...,...,...,...,...,...,...,...
79,38.68,41.52,42.11,43.49,41.45,41.38,41.86,40.89,41.35,3543671.43
80,44.98,48.89,53.64,52.73,50.06,49.78,50.66,49.14,49.88,6166753.57
81,53.06,50.59,54.56,56.11,53.58,53.59,54.48,52.60,53.56,8084530.97
82,52.34,48.89,47.11,42.67,47.75,48.24,48.91,47.08,47.98,7730639.29


## FPT

In [49]:
close = stock[stock_symbols[6]]['close'].values
open = stock[stock_symbols[6]]['open'].values
high = stock[stock_symbols[6]]['high'].values
low = stock[stock_symbols[6]]['low'].values
avg = stock[stock_symbols[6]]['avg'].values
volume = stock[stock_symbols[6]]['volume'].values

In [50]:
reconstruct_close = reconstruct_data(close)

In [51]:
df_fpt = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[6]}'), ('close_week2', f'{stock_symbols[6]}'),
                                  ('close_week3', f'{stock_symbols[6]}'), ('close_week4', f'{stock_symbols[6]}')], dtype = np.dtype(np.float64))
df_fpt.head()

,"(close_week1, FPT)","(close_week2, FPT)","(close_week3, FPT)","(close_week4, FPT)"
0,47.87,50.76,54.64,56.79
1,61.07,62.93,69.21,69.79
2,69.07,69.14,66.93,48.13
3,44.47,45.67,45.29,46.47
4,46.63,49.26,50.60,50.80


In [52]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [53]:
df_fpt[('close_total4week', f'{stock_symbols[6]}')] = mean_close[:len(df_fpt)]
df_fpt[('open_total4week', f'{stock_symbols[6]}')] = mean_open[:len(df_fpt)]
df_fpt[('high_total4week', f'{stock_symbols[6]}')] = mean_high[:len(df_fpt)]
df_fpt[('low_total4week', f'{stock_symbols[6]}')] = mean_low[:len(df_fpt)]
df_fpt[('avg_total4week', f'{stock_symbols[6]}')] = mean_avg[:len(df_fpt)]
df_fpt[('volume_total4week', f'{stock_symbols[6]}')] = mean_volume[:len(df_fpt)]

In [54]:
df_fpt

,"(close_week1, FPT)","(close_week2, FPT)","(close_week3, FPT)","(close_week4, FPT)","(close_total4week, FPT)","(open_total4week, FPT)","(high_total4week, FPT)","(low_total4week, FPT)","(avg_total4week, FPT)","(volume_total4week, FPT)"
0,47.87,50.76,54.64,56.79,52.51,52.23,53.12,51.66,52.39,659624.29
1,61.07,62.93,69.21,69.79,65.75,65.57,66.77,64.46,65.59,1151072.50
2,69.07,69.14,66.93,48.13,63.32,63.61,64.52,62.36,63.33,1168561.79
3,44.47,45.67,45.29,46.47,45.48,45.38,46.02,44.85,45.45,766297.86
4,46.63,49.26,50.60,50.80,49.32,49.18,49.96,48.79,49.35,1107976.43
...,...,...,...,...,...,...,...,...,...,...
84,84.74,86.17,78.34,77.99,81.81,81.65,82.33,81.23,81.77,1129992.86
85,82.81,83.83,83.91,89.59,85.04,84.52,85.68,83.85,84.78,2278878.96
86,97.43,97.73,94.63,93.44,95.81,95.68,96.89,94.42,95.54,3418422.71
87,94.91,89.37,87.69,92.06,91.01,91.49,92.35,90.03,91.13,3525649.18


## HPG

In [55]:
close = stock[stock_symbols[7]]['close'].values
open = stock[stock_symbols[7]]['open'].values
high = stock[stock_symbols[7]]['high'].values
low = stock[stock_symbols[7]]['low'].values
avg = stock[stock_symbols[7]]['avg'].values
volume = stock[stock_symbols[7]]['volume'].values

In [56]:
reconstruct_close = reconstruct_data(close)

In [57]:
df_hpg = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[7]}'), ('close_week2', f'{stock_symbols[7]}'),
                                  ('close_week3', f'{stock_symbols[7]}'), ('close_week4', f'{stock_symbols[7]}')], dtype = np.dtype(np.float64))
df_hpg.head()

,"(close_week1, HPG)","(close_week2, HPG)","(close_week3, HPG)","(close_week4, HPG)"
0,42.29,48.33,47.66,48.44
1,49.13,48.71,50.84,52.36
2,54.64,54.21,50.77,45.81
3,48.60,51.14,49.79,52.29
4,53.64,55.14,57.29,56.14


In [58]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [59]:
df_hpg[('close_total4week', f'{stock_symbols[7]}')] = mean_close[:len(df_hpg)]
df_hpg[('open_total4week', f'{stock_symbols[7]}')] = mean_open[:len(df_hpg)]
df_hpg[('high_total4week', f'{stock_symbols[7]}')] = mean_high[:len(df_hpg)]
df_hpg[('low_total4week', f'{stock_symbols[7]}')] = mean_low[:len(df_hpg)]
df_hpg[('avg_total4week', f'{stock_symbols[7]}')] = mean_avg[:len(df_hpg)]
df_hpg[('volume_total4week', f'{stock_symbols[7]}')] = mean_volume[:len(df_hpg)]

In [60]:
df_hpg

,"(close_week1, HPG)","(close_week2, HPG)","(close_week3, HPG)","(close_week4, HPG)","(close_total4week, HPG)","(open_total4week, HPG)","(high_total4week, HPG)","(low_total4week, HPG)","(avg_total4week, HPG)","(volume_total4week, HPG)"
0,42.29,48.33,47.66,48.44,46.68,46.45,47.30,45.88,46.58,586278.57
1,49.13,48.71,50.84,52.36,50.26,50.09,50.96,49.43,50.21,930927.50
2,54.64,54.21,50.77,45.81,51.36,51.19,52.08,50.15,51.13,775278.57
3,48.60,51.14,49.79,52.29,50.45,50.19,51.09,49.52,50.30,557077.50
4,53.64,55.14,57.29,56.14,55.55,55.25,56.02,54.75,55.42,921949.64
...,...,...,...,...,...,...,...,...,...,...
84,23.53,25.69,26.98,27.38,25.89,25.75,26.11,25.48,25.78,24379576.43
85,28.31,27.39,27.67,26.30,27.42,27.47,27.74,27.08,27.39,27604978.99
86,28.14,28.10,26.76,25.37,27.09,27.13,27.56,26.69,27.12,28598482.39
87,25.04,23.65,25.11,26.87,25.17,25.23,25.61,24.76,25.17,24687150.00


## MSN

In [61]:
close = stock[stock_symbols[8]]['close'].values
open = stock[stock_symbols[8]]['open'].values
high = stock[stock_symbols[8]]['high'].values
low = stock[stock_symbols[8]]['low'].values
avg = stock[stock_symbols[8]]['avg'].values
volume = stock[stock_symbols[8]]['volume'].values

In [62]:
reconstruct_close = reconstruct_data(close)

In [63]:
df_msn = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[8]}'), ('close_week2', f'{stock_symbols[8]}'),
                                  ('close_week3', f'{stock_symbols[8]}'), ('close_week4', f'{stock_symbols[8]}')], dtype = np.dtype(np.float64))
df_msn.head()

,"(close_week1, MSN)","(close_week2, MSN)","(close_week3, MSN)","(close_week4, MSN)"
0,83.36,91.29,93.21,93.07
1,98.00,97.57,104.07,99.07
2,94.64,94.29,94.29,90.50
3,91.00,96.00,98.00,95.07
4,91.71,92.36,91.79,90.29


In [64]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [65]:
df_msn[('close_total4week', f'{stock_symbols[8]}')] = mean_close[:len(df_msn)]
df_msn[('open_total4week', f'{stock_symbols[8]}')] = mean_open[:len(df_msn)]
df_msn[('high_total4week', f'{stock_symbols[8]}')] = mean_high[:len(df_msn)]
df_msn[('low_total4week', f'{stock_symbols[8]}')] = mean_low[:len(df_msn)]
df_msn[('avg_total4week', f'{stock_symbols[8]}')] = mean_avg[:len(df_msn)]
df_msn[('volume_total4week', f'{stock_symbols[8]}')] = mean_volume[:len(df_msn)]

In [66]:
df_msn

,"(close_week1, MSN)","(close_week2, MSN)","(close_week3, MSN)","(close_week4, MSN)","(close_total4week, MSN)","(open_total4week, MSN)","(high_total4week, MSN)","(low_total4week, MSN)","(avg_total4week, MSN)","(volume_total4week, MSN)"
0,83.36,91.29,93.21,93.07,90.23,89.98,91.27,89.11,90.22,243162.50
1,98.00,97.57,104.07,99.07,99.68,99.79,101.32,98.57,99.78,598548.93
2,94.64,94.29,94.29,90.50,93.43,93.48,95.12,91.96,93.42,174805.36
3,91.00,96.00,98.00,95.07,95.02,95.14,96.16,93.96,95.02,156626.43
4,91.71,92.36,91.79,90.29,91.54,91.77,92.32,90.91,91.58,159496.43
...,...,...,...,...,...,...,...,...,...,...
84,76.61,76.66,76.44,80.66,77.59,77.65,78.31,76.91,77.59,2526742.86
85,84.66,87.10,82.14,79.14,83.26,83.37,84.26,82.31,83.23,2388935.77
86,81.24,79.51,75.19,73.87,77.45,77.64,78.72,76.51,77.53,3584410.71
87,70.99,63.59,62.01,63.57,65.04,65.43,66.29,63.90,64.95,2384225.00


## SAB

In [67]:
close = stock[stock_symbols[9]]['close'].values
open = stock[stock_symbols[9]]['open'].values
high = stock[stock_symbols[9]]['high'].values
low = stock[stock_symbols[9]]['low'].values
avg = stock[stock_symbols[9]]['avg'].values
volume = stock[stock_symbols[9]]['volume'].values

In [68]:
reconstruct_close = reconstruct_data(close)

In [69]:
df_sab = pd.DataFrame(reconstruct_close, columns =[('close_week1', f'{stock_symbols[9]}'), ('close_week2', f'{stock_symbols[9]}'),
                                  ('close_week3', f'{stock_symbols[9]}'), ('close_week4', f'{stock_symbols[9]}')], dtype = np.dtype(np.float64))
df_sab.head()

,"(close_week1, SAB)","(close_week2, SAB)","(close_week3, SAB)","(close_week4, SAB)"
0,163.00,207.07,197.51,206.43
1,220.13,216.73,214.94,224.86
2,220.47,209.67,200.86,203.61
3,203.64,200.03,197.60,191.16
4,190.23,191.59,200.59,206.90


In [70]:
mean_close = []
mean_open = []
mean_high = []
mean_low = []
mean_avg = []
mean_volume = []

i = 0
while (i < len(close)):
  mean_close.append(np.round(np.mean(close[i:i+28]), 2))
  mean_open.append(np.round(np.mean(open[i:i+28]), 2))
  mean_high.append(np.round(np.mean(high[i:i+28]), 2))
  mean_low.append(np.round(np.mean(low[i:i+28]), 2))
  mean_avg.append(np.round(np.mean(avg[i:i+28]), 2))
  mean_volume.append(np.round(np.mean(volume[i:i+28]), 2))
  i = i + 28

In [71]:
df_sab[('close_total4week', f'{stock_symbols[9]}')] = mean_close[:len(df_sab)]
df_sab[('open_total4week', f'{stock_symbols[9]}')] = mean_open[:len(df_sab)]
df_sab[('high_total4week', f'{stock_symbols[9]}')] = mean_high[:len(df_sab)]
df_sab[('low_total4week', f'{stock_symbols[9]}')] = mean_low[:len(df_sab)]
df_sab[('avg_total4week', f'{stock_symbols[9]}')] = mean_avg[:len(df_sab)]
df_sab[('volume_total4week', f'{stock_symbols[9]}')] = mean_volume[:len(df_sab)]

In [72]:
df_sab

,"(close_week1, SAB)","(close_week2, SAB)","(close_week3, SAB)","(close_week4, SAB)","(close_total4week, SAB)","(open_total4week, SAB)","(high_total4week, SAB)","(low_total4week, SAB)","(avg_total4week, SAB)","(volume_total4week, SAB)"
0,163.00,207.07,197.51,206.43,193.50,193.66,195.70,189.88,193.21,115343.21
1,220.13,216.73,214.94,224.86,219.16,218.49,220.92,216.68,218.70,68125.36
2,220.47,209.67,200.86,203.61,208.65,209.16,210.87,206.50,208.56,31002.50
3,203.64,200.03,197.60,191.16,198.11,198.66,199.64,197.18,198.16,32071.43
4,190.23,191.59,200.59,206.90,197.32,196.91,198.14,195.04,196.66,39309.29
...,...,...,...,...,...,...,...,...,...,...
58,158.84,155.06,152.77,152.53,154.80,155.36,156.10,154.21,154.89,141939.29
59,156.87,159.81,158.49,153.20,157.09,157.62,158.58,156.20,157.27,248728.57
60,159.26,118.76,76.70,70.53,106.31,106.92,107.93,105.55,106.57,439900.00
61,69.27,67.67,61.64,63.16,65.44,65.74,66.64,64.49,65.50,603457.14


## ALL

In [73]:
df_vic = df_vic.ewm(alpha=0.65).mean()
df_vcb = df_vcb.ewm(alpha=0.65).mean()
df_vnm = df_vnm.ewm(alpha=0.65).mean()
df_gas = df_gas.ewm(alpha=0.65).mean()
df_hvn = df_hvn.ewm(alpha=0.65).mean()
df_mwg = df_mwg.ewm(alpha=0.65).mean()
df_fpt = df_fpt.ewm(alpha=0.65).mean()
df_hpg = df_hpg.ewm(alpha=0.65).mean()
df_msn = df_msn.ewm(alpha=0.65).mean()
df_sab = df_sab.ewm(alpha=0.65).mean()

In [74]:
df_vic[('ema50', f'{stock_symbols[0]}')] = df_vic[('close_total4week', f'{stock_symbols[0]}')] / df_vic[('close_total4week', f'{stock_symbols[0]}')].ewm(50).mean()
df_vic[('ema21', f'{stock_symbols[0]}')] = df_vic[('close_total4week', f'{stock_symbols[0]}')] / df_vic[('close_total4week', f'{stock_symbols[0]}')].ewm(21).mean()
df_vic[('ema14', f'{stock_symbols[0]}')] = df_vic[('close_total4week', f'{stock_symbols[0]}')] / df_vic[('close_total4week', f'{stock_symbols[0]}')].ewm(14).mean()
df_vic[('ema5', f'{stock_symbols[0]}')] = df_vic[('close_total4week', f'{stock_symbols[0]}')] / df_vic[('close_total4week', f'{stock_symbols[0]}')].ewm(5).mean()

In [75]:
df_vcb[('ema50', f'{stock_symbols[1]}')] = df_vcb[('close_total4week', f'{stock_symbols[1]}')] / df_vcb[('close_total4week', f'{stock_symbols[1]}')].ewm(50).mean()
df_vcb[('ema21', f'{stock_symbols[1]}')] = df_vcb[('close_total4week', f'{stock_symbols[1]}')] / df_vcb[('close_total4week', f'{stock_symbols[1]}')].ewm(21).mean()
df_vcb[('ema14', f'{stock_symbols[1]}')] = df_vcb[('close_total4week', f'{stock_symbols[1]}')] / df_vcb[('close_total4week', f'{stock_symbols[1]}')].ewm(14).mean()
df_vcb[('ema5', f'{stock_symbols[1]}')] = df_vcb[('close_total4week', f'{stock_symbols[1]}')] / df_vcb[('close_total4week', f'{stock_symbols[1]}')].ewm(5).mean()

In [76]:
df_vnm[('ema50', f'{stock_symbols[2]}')] = df_vnm[('close_total4week', f'{stock_symbols[2]}')] / df_vnm[('close_total4week', f'{stock_symbols[2]}')].ewm(50).mean()
df_vnm[('ema21', f'{stock_symbols[2]}')] = df_vnm[('close_total4week', f'{stock_symbols[2]}')] / df_vnm[('close_total4week', f'{stock_symbols[2]}')].ewm(21).mean()
df_vnm[('ema14', f'{stock_symbols[2]}')] = df_vnm[('close_total4week', f'{stock_symbols[2]}')] / df_vnm[('close_total4week', f'{stock_symbols[2]}')].ewm(14).mean()
df_vnm[('ema5', f'{stock_symbols[2]}')] = df_vnm[('close_total4week', f'{stock_symbols[2]}')] / df_vnm[('close_total4week', f'{stock_symbols[2]}')].ewm(5).mean()

In [77]:
df_gas[('ema50', f'{stock_symbols[3]}')] = df_gas[('close_total4week', f'{stock_symbols[3]}')] / df_gas[('close_total4week', f'{stock_symbols[3]}')].ewm(50).mean()
df_gas[('ema21', f'{stock_symbols[3]}')] = df_gas[('close_total4week', f'{stock_symbols[3]}')] / df_gas[('close_total4week', f'{stock_symbols[3]}')].ewm(21).mean()
df_gas[('ema14', f'{stock_symbols[3]}')] = df_gas[('close_total4week', f'{stock_symbols[3]}')] / df_gas[('close_total4week', f'{stock_symbols[3]}')].ewm(14).mean()
df_gas[('ema5', f'{stock_symbols[3]}')] = df_gas[('close_total4week', f'{stock_symbols[3]}')] / df_gas[('close_total4week', f'{stock_symbols[3]}')].ewm(5).mean()

In [78]:
df_hvn[('ema50', f'{stock_symbols[4]}')] = df_hvn[('close_total4week', f'{stock_symbols[4]}')] / df_hvn[('close_total4week', f'{stock_symbols[4]}')].ewm(50).mean()
df_hvn[('ema21', f'{stock_symbols[4]}')] = df_hvn[('close_total4week', f'{stock_symbols[4]}')] / df_hvn[('close_total4week', f'{stock_symbols[4]}')].ewm(21).mean()
df_hvn[('ema14', f'{stock_symbols[4]}')] = df_hvn[('close_total4week', f'{stock_symbols[4]}')] / df_hvn[('close_total4week', f'{stock_symbols[4]}')].ewm(14).mean()
df_hvn[('ema5', f'{stock_symbols[4]}')] = df_hvn[('close_total4week', f'{stock_symbols[4]}')] / df_hvn[('close_total4week', f'{stock_symbols[4]}')].ewm(5).mean()

In [79]:
df_mwg[('ema50', f'{stock_symbols[5]}')] = df_mwg[('close_total4week', f'{stock_symbols[5]}')] / df_mwg[('close_total4week', f'{stock_symbols[5]}')].ewm(50).mean()
df_mwg[('ema21', f'{stock_symbols[5]}')] = df_mwg[('close_total4week', f'{stock_symbols[5]}')] / df_mwg[('close_total4week', f'{stock_symbols[5]}')].ewm(21).mean()
df_mwg[('ema14', f'{stock_symbols[5]}')] = df_mwg[('close_total4week', f'{stock_symbols[5]}')] / df_mwg[('close_total4week', f'{stock_symbols[5]}')].ewm(14).mean()
df_mwg[('ema5', f'{stock_symbols[5]}')] = df_mwg[('close_total4week', f'{stock_symbols[5]}')] / df_mwg[('close_total4week', f'{stock_symbols[5]}')].ewm(5).mean()

In [80]:
df_fpt[('ema50', f'{stock_symbols[6]}')] = df_fpt[('close_total4week', f'{stock_symbols[6]}')] / df_fpt[('close_total4week', f'{stock_symbols[6]}')].ewm(50).mean()
df_fpt[('ema21', f'{stock_symbols[6]}')] = df_fpt[('close_total4week', f'{stock_symbols[6]}')] / df_fpt[('close_total4week', f'{stock_symbols[6]}')].ewm(21).mean()
df_fpt[('ema14', f'{stock_symbols[6]}')] = df_fpt[('close_total4week', f'{stock_symbols[6]}')] / df_fpt[('close_total4week', f'{stock_symbols[6]}')].ewm(14).mean()
df_fpt[('ema5', f'{stock_symbols[6]}')] = df_fpt[('close_total4week', f'{stock_symbols[6]}')] / df_fpt[('close_total4week', f'{stock_symbols[6]}')].ewm(5).mean()

In [81]:
df_hpg[('ema50', f'{stock_symbols[7]}')] = df_hpg[('close_total4week', f'{stock_symbols[7]}')] / df_hpg[('close_total4week', f'{stock_symbols[7]}')].ewm(50).mean()
df_hpg[('ema21', f'{stock_symbols[7]}')] = df_hpg[('close_total4week', f'{stock_symbols[7]}')] / df_hpg[('close_total4week', f'{stock_symbols[7]}')].ewm(21).mean()
df_hpg[('ema14', f'{stock_symbols[7]}')] = df_hpg[('close_total4week', f'{stock_symbols[7]}')] / df_hpg[('close_total4week', f'{stock_symbols[7]}')].ewm(14).mean()
df_hpg[('ema5', f'{stock_symbols[7]}')] = df_hpg[('close_total4week', f'{stock_symbols[7]}')] / df_hpg[('close_total4week', f'{stock_symbols[7]}')].ewm(5).mean()

In [82]:
df_msn[('ema50', f'{stock_symbols[8]}')] = df_msn[('close_total4week', f'{stock_symbols[8]}')] / df_msn[('close_total4week', f'{stock_symbols[8]}')].ewm(50).mean()
df_msn[('ema21', f'{stock_symbols[8]}')] = df_msn[('close_total4week', f'{stock_symbols[8]}')] / df_msn[('close_total4week', f'{stock_symbols[8]}')].ewm(21).mean()
df_msn[('ema14', f'{stock_symbols[8]}')] = df_msn[('close_total4week', f'{stock_symbols[8]}')] / df_msn[('close_total4week', f'{stock_symbols[8]}')].ewm(14).mean()
df_msn[('ema5', f'{stock_symbols[8]}')] = df_msn[('close_total4week', f'{stock_symbols[8]}')] / df_msn[('close_total4week', f'{stock_symbols[8]}')].ewm(5).mean()

In [83]:
df_sab[('ema50', f'{stock_symbols[9]}')] = df_sab[('close_total4week', f'{stock_symbols[9]}')] / df_sab[('close_total4week', f'{stock_symbols[9]}')].ewm(50).mean()
df_sab[('ema21', f'{stock_symbols[9]}')] = df_sab[('close_total4week', f'{stock_symbols[9]}')] / df_sab[('close_total4week', f'{stock_symbols[9]}')].ewm(21).mean()
df_sab[('ema14', f'{stock_symbols[9]}')] = df_sab[('close_total4week', f'{stock_symbols[9]}')] / df_sab[('close_total4week', f'{stock_symbols[9]}')].ewm(14).mean()
df_sab[('ema5', f'{stock_symbols[9]}')] = df_sab[('close_total4week', f'{stock_symbols[9]}')] / df_sab[('close_total4week', f'{stock_symbols[9]}')].ewm(5).mean()

In [84]:
def rsi(X, window=14):
  delta = X.diff(1)

  gains = delta.where(delta > 0, 0)
  losses = -delta.where(delta < 0, 0)

  avg_gains = gains.rolling(window=window, min_periods=1).mean()
  avg_losses = losses.rolling(window=window, min_periods=1).mean()

  rs = avg_gains / avg_losses
  rsi = 100 - (100 / (1 + rs))

  return rsi

In [85]:
df_vic[('rsi', f'{stock_symbols[0]}')] = rsi(df_vic[('close_total4week', f'{stock_symbols[0]}')])
df_vcb[('rsi', f'{stock_symbols[1]}')] = rsi(df_vcb[('close_total4week', f'{stock_symbols[1]}')])
df_vnm[('rsi', f'{stock_symbols[2]}')] = rsi(df_vnm[('close_total4week', f'{stock_symbols[2]}')])
df_gas[('rsi', f'{stock_symbols[3]}')] = rsi(df_gas[('close_total4week', f'{stock_symbols[3]}')])
df_hvn[('rsi', f'{stock_symbols[4]}')] = rsi(df_hvn[('close_total4week', f'{stock_symbols[4]}')])
df_mwg[('rsi', f'{stock_symbols[5]}')] = rsi(df_mwg[('close_total4week', f'{stock_symbols[5]}')])
df_fpt[('rsi', f'{stock_symbols[6]}')] = rsi(df_fpt[('close_total4week', f'{stock_symbols[6]}')])
df_hpg[('rsi', f'{stock_symbols[7]}')] = rsi(df_hpg[('close_total4week', f'{stock_symbols[7]}')])
df_msn[('rsi', f'{stock_symbols[8]}')] = rsi(df_msn[('close_total4week', f'{stock_symbols[8]}')])
df_sab[('rsi', f'{stock_symbols[9]}')] = rsi(df_sab[('close_total4week', f'{stock_symbols[9]}')])

In [86]:
def macd(X, short_window=12, long_window=29, signal=9):
  short_ema = X.ewm(span=short_window, adjust=False).mean()

  long_ema = X.ewm(span=long_window, adjust=False).mean()

  macd = short_ema - long_ema

  signal = macd.ewm(span=signal, adjust=False).mean()

  return signal

In [87]:
df_vic[('macd', f'{stock_symbols[0]}')] = macd(df_vic[('close_total4week', f'{stock_symbols[0]}')])
df_vcb[('macd', f'{stock_symbols[1]}')] = macd(df_vcb[('close_total4week', f'{stock_symbols[1]}')])
df_vnm[('macd', f'{stock_symbols[2]}')] = macd(df_vnm[('close_total4week', f'{stock_symbols[2]}')])
df_gas[('macd', f'{stock_symbols[3]}')] = macd(df_gas[('close_total4week', f'{stock_symbols[3]}')])
df_hvn[('macd', f'{stock_symbols[4]}')] = macd(df_hvn[('close_total4week', f'{stock_symbols[4]}')])
df_mwg[('macd', f'{stock_symbols[5]}')] = macd(df_mwg[('close_total4week', f'{stock_symbols[5]}')])
df_fpt[('macd', f'{stock_symbols[6]}')] = macd(df_fpt[('close_total4week', f'{stock_symbols[6]}')])
df_hpg[('macd', f'{stock_symbols[7]}')] = macd(df_hpg[('close_total4week', f'{stock_symbols[7]}')])
df_msn[('macd', f'{stock_symbols[8]}')] = macd(df_msn[('close_total4week', f'{stock_symbols[8]}')])
df_sab[('macd', f'{stock_symbols[9]}')] = macd(df_sab[('close_total4week', f'{stock_symbols[9]}')])

In [88]:
def obv(X, num):
  obv = pd.Series(index=X.index)
  obv.iloc[0] = 0

  for i in range(1, len(X)):
    if (X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i].values > X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values):
      obv.iloc[i] = obv.iloc[i-1] + X[[('volume_total4week', f'{stock_symbols[num]}')]].iloc[i].values
    elif (X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i].values < X[[('close_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values):
      obv.iloc[i] = obv.iloc[i-1] - X[[('volume_total4week', f'{stock_symbols[num]}')]].iloc[i].values
    else:
      obv.iloc[i] = obv.iloc[i-1]

  return obv

In [89]:
df_vic[('obv', f'{stock_symbols[0]}')] = obv(df_vic, 0)
df_vcb[('obv', f'{stock_symbols[1]}')] = obv(df_vcb, 1)
df_vnm[('obv', f'{stock_symbols[2]}')] = obv(df_vnm, 2)
df_gas[('obv', f'{stock_symbols[3]}')] = obv(df_gas, 3)
df_hvn[('obv', f'{stock_symbols[4]}')] = obv(df_hvn, 4)
df_mwg[('obv', f'{stock_symbols[5]}')] = obv(df_mwg, 5)
df_fpt[('obv', f'{stock_symbols[6]}')] = obv(df_fpt, 6)
df_hpg[('obv', f'{stock_symbols[7]}')] = obv(df_hpg, 7)
df_msn[('obv', f'{stock_symbols[8]}')] = obv(df_msn, 8)
df_sab[('obv', f'{stock_symbols[9]}')] = obv(df_sab, 9)

In [90]:
def emv(X, num):
  emv = pd.Series(index=X.index)
  emv.iloc[0] = np.nan

  for i in range(1, len(X)):
    dm = 0.5 * ((X[[('high_total4week', f'{stock_symbols[num]}')]].iloc[i].values + X[[('low_total4week', f'{stock_symbols[num]}')]].iloc[i].values) - (X[[('high_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values + X[[('low_total4week', f'{stock_symbols[num]}')]].iloc[i-1].values))
    br = X[[('volume_total4week', f'{stock_symbols[num]}')]].iloc[i].values / (1000000 * (X[[('high_total4week', f'{stock_symbols[num]}')]].iloc[i].values - X[[('low_total4week', f'{stock_symbols[num]}')]].iloc[i].values))
    emv.iloc[i] = dm / br if br != 0 else 0

  return emv

In [91]:
df_vic[('emv', f'{stock_symbols[0]}')] = emv(df_vic, 0)
df_vcb[('emv', f'{stock_symbols[1]}')] = emv(df_vcb, 1)
df_vnm[('emv', f'{stock_symbols[2]}')] = emv(df_vnm, 2)
df_gas[('emv', f'{stock_symbols[3]}')] = emv(df_gas, 3)
df_hvn[('emv', f'{stock_symbols[4]}')] = emv(df_hvn, 4)
df_mwg[('emv', f'{stock_symbols[5]}')] = emv(df_mwg, 5)
df_fpt[('emv', f'{stock_symbols[6]}')] = emv(df_fpt, 6)
df_hpg[('emv', f'{stock_symbols[7]}')] = emv(df_hpg, 7)
df_msn[('emv', f'{stock_symbols[8]}')] = emv(df_msn, 8)
df_sab[('emv', f'{stock_symbols[9]}')] = emv(df_sab, 9)

In [92]:
def mfi(high, low, close, volume):
  combine_price = (high + low + close) / 3

  raw_money_flow = combine_price * volume

  flow_direction = (combine_price.diff() > 0).astype(int)

  positive_money_flow = flow_direction * raw_money_flow
  negative_money_flow = (1 - flow_direction) * raw_money_flow

  positive = positive_money_flow.rolling(window=14, min_periods=1).sum()
  negative = negative_money_flow.rolling(window=14, min_periods=1).sum()

  mf = positive / negative
  mfi = 100 - (100 / (1 + mf))

  return mfi

In [93]:
df_vic[('mfi', f'{stock_symbols[0]}')] = mfi(df_vic[('high_total4week', f'{stock_symbols[0]}')], df_vic[('low_total4week', f'{stock_symbols[0]}')],
                                             df_vic[('close_total4week', f'{stock_symbols[0]}')], df_vic[('volume_total4week', f'{stock_symbols[0]}')])
df_vcb[('mfi', f'{stock_symbols[1]}')] = mfi(df_vcb[('high_total4week', f'{stock_symbols[1]}')], df_vcb[('low_total4week', f'{stock_symbols[1]}')],
                                             df_vcb[('close_total4week', f'{stock_symbols[1]}')], df_vcb[('volume_total4week', f'{stock_symbols[1]}')])
df_vnm[('mfi', f'{stock_symbols[2]}')] = mfi(df_vnm[('high_total4week', f'{stock_symbols[2]}')], df_vnm[('low_total4week', f'{stock_symbols[2]}')],
                                             df_vnm[('close_total4week', f'{stock_symbols[2]}')], df_vnm[('volume_total4week', f'{stock_symbols[2]}')])
df_gas[('mfi', f'{stock_symbols[3]}')] = mfi(df_gas[('high_total4week', f'{stock_symbols[3]}')], df_gas[('low_total4week', f'{stock_symbols[3]}')],
                                             df_gas[('close_total4week', f'{stock_symbols[3]}')], df_gas[('volume_total4week', f'{stock_symbols[3]}')])
df_hvn[('mfi', f'{stock_symbols[4]}')] = mfi(df_hvn[('high_total4week', f'{stock_symbols[4]}')], df_hvn[('low_total4week', f'{stock_symbols[4]}')],
                                             df_hvn[('close_total4week', f'{stock_symbols[4]}')], df_hvn[('volume_total4week', f'{stock_symbols[4]}')])
df_mwg[('mfi', f'{stock_symbols[5]}')] = mfi(df_mwg[('high_total4week', f'{stock_symbols[5]}')], df_mwg[('low_total4week', f'{stock_symbols[5]}')],
                                             df_mwg[('close_total4week', f'{stock_symbols[5]}')], df_mwg[('volume_total4week', f'{stock_symbols[5]}')])
df_fpt[('mfi', f'{stock_symbols[6]}')] = mfi(df_fpt[('high_total4week', f'{stock_symbols[6]}')], df_fpt[('low_total4week', f'{stock_symbols[6]}')],
                                             df_fpt[('close_total4week', f'{stock_symbols[6]}')], df_fpt[('volume_total4week', f'{stock_symbols[6]}')])
df_hpg[('mfi', f'{stock_symbols[7]}')] = mfi(df_hpg[('high_total4week', f'{stock_symbols[7]}')], df_hpg[('low_total4week', f'{stock_symbols[7]}')],
                                             df_hpg[('close_total4week', f'{stock_symbols[7]}')], df_hpg[('volume_total4week', f'{stock_symbols[7]}')])
df_msn[('mfi', f'{stock_symbols[8]}')] = mfi(df_msn[('high_total4week', f'{stock_symbols[8]}')], df_msn[('low_total4week', f'{stock_symbols[8]}')],
                                             df_msn[('close_total4week', f'{stock_symbols[8]}')], df_msn[('volume_total4week', f'{stock_symbols[8]}')])
df_sab[('mfi', f'{stock_symbols[9]}')] = mfi(df_sab[('high_total4week', f'{stock_symbols[9]}')], df_sab[('low_total4week', f'{stock_symbols[9]}')],
                                             df_sab[('close_total4week', f'{stock_symbols[9]}')], df_sab[('volume_total4week', f'{stock_symbols[9]}')])

In [94]:
df_vic[[('volume_total4week', f'{stock_symbols[0]}')]] = df_vic[[('volume_total4week', f'{stock_symbols[0]}')]] / df_vic[[('volume_total4week', f'{stock_symbols[0]}')]].ewm(5).mean()
df_vcb[[('volume_total4week', f'{stock_symbols[1]}')]] = df_vcb[[('volume_total4week', f'{stock_symbols[1]}')]] / df_vcb[[('volume_total4week', f'{stock_symbols[1]}')]].ewm(5).mean()
df_vnm[[('volume_total4week', f'{stock_symbols[2]}')]] = df_vnm[[('volume_total4week', f'{stock_symbols[2]}')]] / df_vnm[[('volume_total4week', f'{stock_symbols[2]}')]].ewm(5).mean()
df_gas[[('volume_total4week', f'{stock_symbols[3]}')]] = df_gas[[('volume_total4week', f'{stock_symbols[3]}')]] / df_gas[[('volume_total4week', f'{stock_symbols[3]}')]].ewm(5).mean()
df_hvn[[('volume_total4week', f'{stock_symbols[4]}')]] = df_hvn[[('volume_total4week', f'{stock_symbols[4]}')]] / df_hvn[[('volume_total4week', f'{stock_symbols[4]}')]].ewm(5).mean()
df_mwg[[('volume_total4week', f'{stock_symbols[5]}')]] = df_mwg[[('volume_total4week', f'{stock_symbols[5]}')]] / df_mwg[[('volume_total4week', f'{stock_symbols[5]}')]].ewm(5).mean()
df_fpt[[('volume_total4week', f'{stock_symbols[6]}')]] = df_fpt[[('volume_total4week', f'{stock_symbols[6]}')]] / df_fpt[[('volume_total4week', f'{stock_symbols[6]}')]].ewm(5).mean()
df_hpg[[('volume_total4week', f'{stock_symbols[7]}')]] = df_hpg[[('volume_total4week', f'{stock_symbols[7]}')]] / df_hpg[[('volume_total4week', f'{stock_symbols[7]}')]].ewm(5).mean()
df_msn[[('volume_total4week', f'{stock_symbols[8]}')]] = df_msn[[('volume_total4week', f'{stock_symbols[8]}')]] / df_msn[[('volume_total4week', f'{stock_symbols[8]}')]].ewm(5).mean()
df_sab[[('volume_total4week', f'{stock_symbols[9]}')]] = df_sab[[('volume_total4week', f'{stock_symbols[9]}')]] / df_sab[[('volume_total4week', f'{stock_symbols[9]}')]].ewm(5).mean()

In [95]:
df_vic

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)","(close_total4week, VIC)","(open_total4week, VIC)","(high_total4week, VIC)","(low_total4week, VIC)","(avg_total4week, VIC)","(volume_total4week, VIC)","(ema50, VIC)","(ema21, VIC)","(ema14, VIC)","(ema5, VIC)","(rsi, VIC)","(macd, VIC)","(obv, VIC)","(emv, VIC)","(mfi, VIC)"
0,70.430000,72.930000,75.930000,77.430000,74.180000,74.120000,74.820000,73.200000,74.030000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,0.000000,0.000000e+00,NaN,0.000000
1,75.822593,74.196667,76.826296,76.000370,75.713333,75.971852,76.516296,74.985185,75.726296,1.150999,1.010127,1.009989,1.009873,1.009291,100.000000,0.026735,3.647390e+05,7.307327,58.973335
2,73.471087,69.166672,69.426435,67.463616,69.877453,70.021528,70.732733,69.019015,69.800119,1.368741,0.954240,0.954749,0.955183,0.957443,20.807286,-0.032791,-2.069919e+05,-17.609446,31.828188
3,68.587090,67.875459,66.643998,65.224160,67.081147,67.169740,67.800962,66.340460,67.035048,0.898409,0.936291,0.937488,0.938503,0.943696,15.083668,-0.192514,-5.649965e+05,-11.443835,24.930677
4,65.681219,68.747491,68.745437,69.788931,68.243506,68.319950,68.956908,67.562077,68.234064,1.010669,0.962016,0.963496,0.964737,0.970856,23.796977,-0.386852,-1.605910e+05,4.100213,39.901517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,52.655544,52.625601,51.504168,52.236176,52.250608,52.414827,52.786729,51.897829,52.282568,1.138483,0.642770,0.642366,0.668746,0.829014,4.799753,-10.151367,-2.210702e+07,-0.144309,6.459576
85,52.255441,58.744960,64.351959,60.168662,58.880213,59.321189,59.945355,58.236740,59.034399,2.647706,0.729144,0.733245,0.766217,0.944559,15.869197,-10.788884,-1.229487e+07,1.175181,26.791567
86,57.939404,56.479736,53.235686,50.692532,54.590074,55.244916,55.859874,53.981359,54.832540,2.493764,0.681286,0.690041,0.724410,0.894257,18.055298,-11.276385,-2.547375e+07,-0.594452,23.585122
87,49.437791,47.587908,45.665990,46.790886,47.368526,47.636721,48.137956,46.635476,47.349389,1.517901,0.596962,0.610073,0.644578,0.806057,17.016013,-11.747391,-3.442236e+07,-1.264950,22.776413


In [96]:
def closediff(x, y):
  res = []

  for i in range(len(x)):
    res.append(((y[i]-x[i]) / x[i])*100)

  return res

In [97]:
close_week1 = df_vic[('close_week1', f'{stock_symbols[0]}')].values
close_week2 = df_vic[('close_week2', f'{stock_symbols[0]}')].values
close_week3 = df_vic[('close_week3', f'{stock_symbols[0]}')].values
close_week4 = df_vic[('close_week4', f'{stock_symbols[0]}')].values

df_vic[('closediff_week2_1', f'{stock_symbols[0]}')] = closediff(close_week1, close_week2)
df_vic[('closediff_week3_1', f'{stock_symbols[0]}')] = closediff(close_week1, close_week3)
df_vic[('closediff_week4_1', f'{stock_symbols[0]}')] = closediff(close_week1, close_week4)
df_vic[('closediff_week3_2', f'{stock_symbols[0]}')] = closediff(close_week2, close_week3)
df_vic[('closediff_week4_2', f'{stock_symbols[0]}')] = closediff(close_week2, close_week4)
df_vic[('closediff_week4_3', f'{stock_symbols[0]}')] = closediff(close_week3, close_week4)

In [98]:
close_week1 = df_vcb[('close_week1', f'{stock_symbols[1]}')].values
close_week2 = df_vcb[('close_week2', f'{stock_symbols[1]}')].values
close_week3 = df_vcb[('close_week3', f'{stock_symbols[1]}')].values
close_week4 = df_vcb[('close_week4', f'{stock_symbols[1]}')].values

df_vcb[('closediff_week2_1', f'{stock_symbols[1]}')] = closediff(close_week1, close_week2)
df_vcb[('closediff_week3_1', f'{stock_symbols[1]}')] = closediff(close_week1, close_week3)
df_vcb[('closediff_week4_1', f'{stock_symbols[1]}')] = closediff(close_week1, close_week4)
df_vcb[('closediff_week3_2', f'{stock_symbols[1]}')] = closediff(close_week2, close_week3)
df_vcb[('closediff_week4_2', f'{stock_symbols[1]}')] = closediff(close_week2, close_week4)
df_vcb[('closediff_week4_3', f'{stock_symbols[1]}')] = closediff(close_week3, close_week4)

In [99]:
close_week1 = df_vnm[('close_week1', f'{stock_symbols[2]}')].values
close_week2 = df_vnm[('close_week2', f'{stock_symbols[2]}')].values
close_week3 = df_vnm[('close_week3', f'{stock_symbols[2]}')].values
close_week4 = df_vnm[('close_week4', f'{stock_symbols[2]}')].values

df_vnm[('closediff_week2_1', f'{stock_symbols[2]}')] = closediff(close_week1, close_week2)
df_vnm[('closediff_week3_1', f'{stock_symbols[2]}')] = closediff(close_week1, close_week3)
df_vnm[('closediff_week4_1', f'{stock_symbols[2]}')] = closediff(close_week1, close_week4)
df_vnm[('closediff_week3_2', f'{stock_symbols[2]}')] = closediff(close_week2, close_week3)
df_vnm[('closediff_week4_2', f'{stock_symbols[2]}')] = closediff(close_week2, close_week4)
df_vnm[('closediff_week4_3', f'{stock_symbols[2]}')] = closediff(close_week3, close_week4)

In [100]:
close_week1 = df_gas[('close_week1', f'{stock_symbols[3]}')].values
close_week2 = df_gas[('close_week2', f'{stock_symbols[3]}')].values
close_week3 = df_gas[('close_week3', f'{stock_symbols[3]}')].values
close_week4 = df_gas[('close_week4', f'{stock_symbols[3]}')].values

df_gas[('closediff_week2_1', f'{stock_symbols[3]}')] = closediff(close_week1, close_week2)
df_gas[('closediff_week3_1', f'{stock_symbols[3]}')] = closediff(close_week1, close_week3)
df_gas[('closediff_week4_1', f'{stock_symbols[3]}')] = closediff(close_week1, close_week4)
df_gas[('closediff_week3_2', f'{stock_symbols[3]}')] = closediff(close_week2, close_week3)
df_gas[('closediff_week4_2', f'{stock_symbols[3]}')] = closediff(close_week2, close_week4)
df_gas[('closediff_week4_3', f'{stock_symbols[3]}')] = closediff(close_week3, close_week4)

In [101]:
close_week1 = df_hvn[('close_week1', f'{stock_symbols[4]}')].values
close_week2 = df_hvn[('close_week2', f'{stock_symbols[4]}')].values
close_week3 = df_hvn[('close_week3', f'{stock_symbols[4]}')].values
close_week4 = df_hvn[('close_week4', f'{stock_symbols[4]}')].values

df_hvn[('closediff_week2_1', f'{stock_symbols[4]}')] = closediff(close_week1, close_week2)
df_hvn[('closediff_week3_1', f'{stock_symbols[4]}')] = closediff(close_week1, close_week3)
df_hvn[('closediff_week4_1', f'{stock_symbols[4]}')] = closediff(close_week1, close_week4)
df_hvn[('closediff_week3_2', f'{stock_symbols[4]}')] = closediff(close_week2, close_week3)
df_hvn[('closediff_week4_2', f'{stock_symbols[4]}')] = closediff(close_week2, close_week4)
df_hvn[('closediff_week4_3', f'{stock_symbols[4]}')] = closediff(close_week3, close_week4)

In [102]:
close_week1 = df_mwg[('close_week1', f'{stock_symbols[5]}')].values
close_week2 = df_mwg[('close_week2', f'{stock_symbols[5]}')].values
close_week3 = df_mwg[('close_week3', f'{stock_symbols[5]}')].values
close_week4 = df_mwg[('close_week4', f'{stock_symbols[5]}')].values

df_mwg[('closediff_week2_1', f'{stock_symbols[5]}')] = closediff(close_week1, close_week2)
df_mwg[('closediff_week3_1', f'{stock_symbols[5]}')] = closediff(close_week1, close_week3)
df_mwg[('closediff_week4_1', f'{stock_symbols[5]}')] = closediff(close_week1, close_week4)
df_mwg[('closediff_week3_2', f'{stock_symbols[5]}')] = closediff(close_week2, close_week3)
df_mwg[('closediff_week4_2', f'{stock_symbols[5]}')] = closediff(close_week2, close_week4)
df_mwg[('closediff_week4_3', f'{stock_symbols[5]}')] = closediff(close_week3, close_week4)

In [103]:
close_week1 = df_fpt[('close_week1', f'{stock_symbols[6]}')].values
close_week2 = df_fpt[('close_week2', f'{stock_symbols[6]}')].values
close_week3 = df_fpt[('close_week3', f'{stock_symbols[6]}')].values
close_week4 = df_fpt[('close_week4', f'{stock_symbols[6]}')].values

df_fpt[('closediff_week2_1', f'{stock_symbols[6]}')] = closediff(close_week1, close_week2)
df_fpt[('closediff_week3_1', f'{stock_symbols[6]}')] = closediff(close_week1, close_week3)
df_fpt[('closediff_week4_1', f'{stock_symbols[6]}')] = closediff(close_week1, close_week4)
df_fpt[('closediff_week3_2', f'{stock_symbols[6]}')] = closediff(close_week2, close_week3)
df_fpt[('closediff_week4_2', f'{stock_symbols[6]}')] = closediff(close_week2, close_week4)
df_fpt[('closediff_week4_3', f'{stock_symbols[6]}')] = closediff(close_week3, close_week4)

In [104]:
close_week1 = df_hpg[('close_week1', f'{stock_symbols[7]}')].values
close_week2 = df_hpg[('close_week2', f'{stock_symbols[7]}')].values
close_week3 = df_hpg[('close_week3', f'{stock_symbols[7]}')].values
close_week4 = df_hpg[('close_week4', f'{stock_symbols[7]}')].values

df_hpg[('closediff_week2_1', f'{stock_symbols[7]}')] = closediff(close_week1, close_week2)
df_hpg[('closediff_week3_1', f'{stock_symbols[7]}')] = closediff(close_week1, close_week3)
df_hpg[('closediff_week4_1', f'{stock_symbols[7]}')] = closediff(close_week1, close_week4)
df_hpg[('closediff_week3_2', f'{stock_symbols[7]}')] = closediff(close_week2, close_week3)
df_hpg[('closediff_week4_2', f'{stock_symbols[7]}')] = closediff(close_week2, close_week4)
df_hpg[('closediff_week4_3', f'{stock_symbols[7]}')] = closediff(close_week3, close_week4)

In [105]:
close_week1 = df_msn[('close_week1', f'{stock_symbols[8]}')].values
close_week2 = df_msn[('close_week2', f'{stock_symbols[8]}')].values
close_week3 = df_msn[('close_week3', f'{stock_symbols[8]}')].values
close_week4 = df_msn[('close_week4', f'{stock_symbols[8]}')].values

df_msn[('closediff_week2_1', f'{stock_symbols[8]}')] = closediff(close_week1, close_week2)
df_msn[('closediff_week3_1', f'{stock_symbols[8]}')] = closediff(close_week1, close_week3)
df_msn[('closediff_week4_1', f'{stock_symbols[8]}')] = closediff(close_week1, close_week4)
df_msn[('closediff_week3_2', f'{stock_symbols[8]}')] = closediff(close_week2, close_week3)
df_msn[('closediff_week4_2', f'{stock_symbols[8]}')] = closediff(close_week2, close_week4)
df_msn[('closediff_week4_3', f'{stock_symbols[8]}')] = closediff(close_week3, close_week4)

In [106]:
close_week1 = df_sab[('close_week1', f'{stock_symbols[9]}')].values
close_week2 = df_sab[('close_week2', f'{stock_symbols[9]}')].values
close_week3 = df_sab[('close_week3', f'{stock_symbols[9]}')].values
close_week4 = df_sab[('close_week4', f'{stock_symbols[9]}')].values

df_sab[('closediff_week2_1', f'{stock_symbols[9]}')] = closediff(close_week1, close_week2)
df_sab[('closediff_week3_1', f'{stock_symbols[9]}')] = closediff(close_week1, close_week3)
df_sab[('closediff_week4_1', f'{stock_symbols[9]}')] = closediff(close_week1, close_week4)
df_sab[('closediff_week3_2', f'{stock_symbols[9]}')] = closediff(close_week2, close_week3)
df_sab[('closediff_week4_2', f'{stock_symbols[9]}')] = closediff(close_week2, close_week4)
df_sab[('closediff_week4_3', f'{stock_symbols[9]}')] = closediff(close_week3, close_week4)

In [107]:
df_vic

,"(close_week1, VIC)","(close_week2, VIC)","(close_week3, VIC)","(close_week4, VIC)","(close_total4week, VIC)","(open_total4week, VIC)","(high_total4week, VIC)","(low_total4week, VIC)","(avg_total4week, VIC)","(volume_total4week, VIC)",...,"(macd, VIC)","(obv, VIC)","(emv, VIC)","(mfi, VIC)","(closediff_week2_1, VIC)","(closediff_week3_1, VIC)","(closediff_week4_1, VIC)","(closediff_week3_2, VIC)","(closediff_week4_2, VIC)","(closediff_week4_3, VIC)"
0,70.430000,72.930000,75.930000,77.430000,74.180000,74.120000,74.820000,73.200000,74.030000,1.000000,...,0.000000,0.000000e+00,NaN,0.000000,3.549624,7.809172,9.938946,4.113534,6.170300,1.975504
1,75.822593,74.196667,76.826296,76.000370,75.713333,75.971852,76.516296,74.985185,75.726296,1.150999,...,0.026735,3.647390e+05,7.307327,58.973335,-2.144382,1.323753,0.234465,3.544134,2.430977,-1.075056
2,73.471087,69.166672,69.426435,67.463616,69.877453,70.021528,70.732733,69.019015,69.800119,1.368741,...,-0.032791,-2.069919e+05,-17.609446,31.828188,-5.858651,-5.505093,-8.176645,0.375560,-2.462249,-2.827192
3,68.587090,67.875459,66.643998,65.224160,67.081147,67.169740,67.800962,66.340460,67.035048,0.898409,...,-0.192514,-5.649965e+05,-11.443835,24.930677,-1.037558,-2.833028,-4.903153,-1.814295,-3.906123,-2.130482
4,65.681219,68.747491,68.745437,69.788931,68.243506,68.319950,68.956908,67.562077,68.234064,1.010669,...,-0.386852,-1.605910e+05,4.100213,39.901517,4.668414,4.665287,6.254013,-0.002988,1.514877,1.517911
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,52.655544,52.625601,51.504168,52.236176,52.250608,52.414827,52.786729,51.897829,52.282568,1.138483,...,-10.151367,-2.210702e+07,-0.144309,6.459576,-0.056867,-2.186619,-0.796437,-2.130964,-0.739991,1.421260
85,52.255441,58.744960,64.351959,60.168662,58.880213,59.321189,59.945355,58.236740,59.034399,2.647706,...,-10.788884,-1.229487e+07,1.175181,26.791567,12.418841,23.148821,15.143344,9.544646,2.423529,-6.500652
86,57.939404,56.479736,53.235686,50.692532,54.590074,55.244916,55.859874,53.981359,54.832540,2.493764,...,-11.276385,-2.547375e+07,-0.594452,23.585122,-2.519301,-8.118341,-12.507675,-5.743742,-10.246515,-4.777160
87,49.437791,47.587908,45.665990,46.790886,47.368526,47.636721,48.137956,46.635476,47.349389,1.517901,...,-11.747391,-3.442236e+07,-1.264950,22.776413,-3.741842,-7.629389,-5.354012,-4.038668,-1.674841,2.463313


In [108]:
def total_closediff(a, b, c, d, e, f):
  res = []

  for i in range(len(a)):
    res.append(((a[i]+b[i]+c[i]+d[i]+e[i]+f[i])/6))

  return res

In [109]:
df_vic[('total_closediff', f'{stock_symbols[0]}')] = total_closediff(df_vic[('closediff_week2_1', f'{stock_symbols[0]}')], df_vic[('closediff_week3_1', f'{stock_symbols[0]}')],
                                                                     df_vic[('closediff_week4_1', f'{stock_symbols[0]}')], df_vic[('closediff_week3_2', f'{stock_symbols[0]}')],
                                                                     df_vic[('closediff_week4_2', f'{stock_symbols[0]}')], df_vic[('closediff_week4_3', f'{stock_symbols[0]}')])

In [110]:
df_vcb[('total_closediff', f'{stock_symbols[1]}')] = total_closediff(df_vcb[('closediff_week2_1', f'{stock_symbols[1]}')], df_vcb[('closediff_week3_1', f'{stock_symbols[1]}')],
                                                                     df_vcb[('closediff_week4_1', f'{stock_symbols[1]}')], df_vcb[('closediff_week3_2', f'{stock_symbols[1]}')],
                                                                     df_vcb[('closediff_week4_2', f'{stock_symbols[1]}')], df_vcb[('closediff_week4_3', f'{stock_symbols[1]}')])

In [111]:
df_vnm[('total_closediff', f'{stock_symbols[2]}')] = total_closediff(df_vnm[('closediff_week2_1', f'{stock_symbols[2]}')], df_vnm[('closediff_week3_1', f'{stock_symbols[2]}')],
                                                                     df_vnm[('closediff_week4_1', f'{stock_symbols[2]}')], df_vnm[('closediff_week3_2', f'{stock_symbols[2]}')],
                                                                     df_vnm[('closediff_week4_2', f'{stock_symbols[2]}')], df_vnm[('closediff_week4_3', f'{stock_symbols[2]}')])

In [112]:
df_gas[('total_closediff', f'{stock_symbols[3]}')] = total_closediff(df_gas[('closediff_week2_1', f'{stock_symbols[3]}')], df_gas[('closediff_week3_1', f'{stock_symbols[3]}')],
                                                                     df_gas[('closediff_week4_1', f'{stock_symbols[3]}')], df_gas[('closediff_week3_2', f'{stock_symbols[3]}')],
                                                                     df_gas[('closediff_week4_2', f'{stock_symbols[3]}')], df_gas[('closediff_week4_3', f'{stock_symbols[3]}')])

In [113]:
df_hvn[('total_closediff', f'{stock_symbols[4]}')] = total_closediff(df_hvn[('closediff_week2_1', f'{stock_symbols[4]}')], df_hvn[('closediff_week3_1', f'{stock_symbols[4]}')],
                                                                     df_hvn[('closediff_week4_1', f'{stock_symbols[4]}')], df_hvn[('closediff_week3_2', f'{stock_symbols[4]}')],
                                                                     df_hvn[('closediff_week4_2', f'{stock_symbols[4]}')], df_hvn[('closediff_week4_3', f'{stock_symbols[4]}')])

In [114]:
df_mwg[('total_closediff', f'{stock_symbols[5]}')] = total_closediff(df_mwg[('closediff_week2_1', f'{stock_symbols[5]}')], df_mwg[('closediff_week3_1', f'{stock_symbols[5]}')],
                                                                     df_mwg[('closediff_week4_1', f'{stock_symbols[5]}')], df_mwg[('closediff_week3_2', f'{stock_symbols[5]}')],
                                                                     df_mwg[('closediff_week4_2', f'{stock_symbols[5]}')], df_mwg[('closediff_week4_3', f'{stock_symbols[5]}')])

In [115]:
df_fpt[('total_closediff', f'{stock_symbols[6]}')] = total_closediff(df_fpt[('closediff_week2_1', f'{stock_symbols[6]}')], df_fpt[('closediff_week3_1', f'{stock_symbols[6]}')],
                                                                     df_fpt[('closediff_week4_1', f'{stock_symbols[6]}')], df_fpt[('closediff_week3_2', f'{stock_symbols[6]}')],
                                                                     df_fpt[('closediff_week4_2', f'{stock_symbols[6]}')], df_fpt[('closediff_week4_3', f'{stock_symbols[6]}')])

In [116]:
df_hpg[('total_closediff', f'{stock_symbols[7]}')] = total_closediff(df_hpg[('closediff_week2_1', f'{stock_symbols[7]}')], df_hpg[('closediff_week3_1', f'{stock_symbols[7]}')],
                                                                     df_hpg[('closediff_week4_1', f'{stock_symbols[7]}')], df_hpg[('closediff_week3_2', f'{stock_symbols[7]}')],
                                                                     df_hpg[('closediff_week4_2', f'{stock_symbols[7]}')], df_hpg[('closediff_week4_3', f'{stock_symbols[7]}')])

In [117]:
df_msn[('total_closediff', f'{stock_symbols[8]}')] = total_closediff(df_msn[('closediff_week2_1', f'{stock_symbols[8]}')], df_msn[('closediff_week3_1', f'{stock_symbols[8]}')],
                                                                     df_msn[('closediff_week4_1', f'{stock_symbols[8]}')], df_msn[('closediff_week3_2', f'{stock_symbols[8]}')],
                                                                     df_msn[('closediff_week4_2', f'{stock_symbols[8]}')], df_msn[('closediff_week4_3', f'{stock_symbols[8]}')])

In [118]:
df_sab[('total_closediff', f'{stock_symbols[9]}')] = total_closediff(df_sab[('closediff_week2_1', f'{stock_symbols[9]}')], df_sab[('closediff_week3_1', f'{stock_symbols[9]}')],
                                                                     df_sab[('closediff_week4_1', f'{stock_symbols[9]}')], df_sab[('closediff_week3_2', f'{stock_symbols[9]}')],
                                                                     df_sab[('closediff_week4_2', f'{stock_symbols[9]}')], df_sab[('closediff_week4_3', f'{stock_symbols[9]}')])

In [119]:
df_vic.replace(0, np.nan, inplace=True)
df_vic = df_vic.dropna()

In [120]:
df_vcb.replace(0, np.nan, inplace=True)
df_vcb = df_vcb.dropna()

In [121]:
df_vnm.replace(0, np.nan, inplace=True)
df_vnm = df_vnm.dropna()

In [122]:
df_gas.replace(0, np.nan, inplace=True)
df_gas = df_gas.dropna()

In [123]:
df_hvn.replace(0, np.nan, inplace=True)
df_hvn = df_hvn.dropna()

In [124]:
df_mwg.replace(0, np.nan, inplace=True)
df_mwg = df_mwg.dropna()

In [125]:
df_fpt.replace(0, np.nan, inplace=True)
df_fpt = df_fpt.dropna()

In [126]:
df_hpg.replace(0, np.nan, inplace=True)
df_hpg = df_hpg.dropna()

In [127]:
df_msn.replace(0, np.nan, inplace=True)
df_msn = df_msn.dropna()

In [128]:
df_sab.replace(0, np.nan, inplace=True)
df_sab = df_sab.dropna()

In [132]:
df_vic[('trend', f'{stock_symbols[0]}')] = (df_vic[('total_closediff', f'{stock_symbols[0]}')].iloc[:] > 0).astype(int)
df_vcb[('trend', f'{stock_symbols[1]}')] = (df_vcb[('total_closediff', f'{stock_symbols[1]}')].iloc[:] > 0).astype(int)
df_vnm[('trend', f'{stock_symbols[2]}')] = (df_vnm[('total_closediff', f'{stock_symbols[2]}')].iloc[:] > 0).astype(int)
df_gas[('trend', f'{stock_symbols[3]}')] = (df_gas[('total_closediff', f'{stock_symbols[3]}')].iloc[:] > 0).astype(int)
df_hvn[('trend', f'{stock_symbols[4]}')] = (df_hvn[('total_closediff', f'{stock_symbols[4]}')].iloc[:] > 0).astype(int)
df_mwg[('trend', f'{stock_symbols[5]}')] = (df_mwg[('total_closediff', f'{stock_symbols[5]}')].iloc[:] > 0).astype(int)
df_fpt[('trend', f'{stock_symbols[6]}')] = (df_fpt[('total_closediff', f'{stock_symbols[6]}')].iloc[:] > 0).astype(int)
df_hpg[('trend', f'{stock_symbols[7]}')] = (df_hpg[('total_closediff', f'{stock_symbols[7]}')].iloc[:] > 0).astype(int)
df_msn[('trend', f'{stock_symbols[8]}')] = (df_msn[('total_closediff', f'{stock_symbols[8]}')].iloc[:] > 0).astype(int)
df_sab[('trend', f'{stock_symbols[9]}')] = (df_sab[('total_closediff', f'{stock_symbols[9]}')].iloc[:] > 0).astype(int)

<ipython-input-132-fc86b30681a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vic[('trend', f'{stock_symbols[0]}')] = (df_vic[('total_closediff', f'{stock_symbols[0]}')].iloc[:] > 0).astype(int)


In [133]:
df_vic

,"(ema50, VIC)","(ema21, VIC)","(ema14, VIC)","(ema5, VIC)","(volume_total4week, VIC)","(rsi, VIC)","(macd, VIC)","(obv, VIC)","(emv, VIC)","(mfi, VIC)","(closediff_week2_1, VIC)","(closediff_week3_1, VIC)","(closediff_week4_1, VIC)","(closediff_week3_2, VIC)","(closediff_week4_2, VIC)","(closediff_week4_3, VIC)","(total_closediff, VIC)","(trend, VIC)"
1,1.010127,1.009989,1.009873,1.009291,1.150999,100.000000,0.026735,3.647390e+05,7.307327,58.973335,-2.144382,1.323753,0.234465,3.544134,2.430977,-1.075056,0.718982,1
2,0.954240,0.954749,0.955183,0.957443,1.368741,20.807286,-0.032791,-2.069919e+05,-17.609446,31.828188,-5.858651,-5.505093,-8.176645,0.375560,-2.462249,-2.827192,-4.075712,0
3,0.936291,0.937488,0.938503,0.943696,0.898409,15.083668,-0.192514,-5.649965e+05,-11.443835,24.930677,-1.037558,-2.833028,-4.903153,-1.814295,-3.906123,-2.130482,-2.770773,0
4,0.962016,0.963496,0.964737,0.970856,1.010669,23.796977,-0.386852,-1.605910e+05,4.100213,39.901517,4.668414,4.665287,6.254013,-0.002988,1.514877,1.517911,3.102919,1
5,0.976855,0.978402,0.979680,0.985704,2.112397,28.315488,-0.580049,1.185264e+06,0.883687,64.022788,2.558200,-7.532955,-16.480113,-9.839443,-18.563423,-9.676050,-9.922297,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,0.642770,0.642366,0.668746,0.829014,1.138483,4.799753,-10.151367,-2.210702e+07,-0.144309,6.459576,-0.056867,-2.186619,-0.796437,-2.130964,-0.739991,1.421260,-0.748270,0
85,0.729144,0.733245,0.766217,0.944559,2.647706,15.869197,-10.788884,-1.229487e+07,1.175181,26.791567,12.418841,23.148821,15.143344,9.544646,2.423529,-6.500652,9.363088,1
86,0.681286,0.690041,0.724410,0.894257,2.493764,18.055298,-11.276385,-2.547375e+07,-0.594452,23.585122,-2.519301,-8.118341,-12.507675,-5.743742,-10.246515,-4.777160,-7.318789,0
87,0.596962,0.610073,0.644578,0.806057,1.517901,17.016013,-11.747391,-3.442236e+07,-1.264950,22.776413,-3.741842,-7.629389,-5.354012,-4.038668,-1.674841,2.463313,-3.329240,0


In [134]:
df_vic = df_vic[[('ema50', f'{stock_symbols[0]}'), ('ema21', f'{stock_symbols[0]}'), ('ema14', f'{stock_symbols[0]}'), ('ema5', f'{stock_symbols[0]}'), ('volume_total4week', f'{stock_symbols[0]}'),
                 ('rsi', f'{stock_symbols[0]}'), ('macd', f'{stock_symbols[0]}'), ('obv', f'{stock_symbols[0]}'), ('emv', f'{stock_symbols[0]}'),
                 ('mfi', f'{stock_symbols[0]}'), ('closediff_week2_1', f'{stock_symbols[0]}'), ('closediff_week3_1', f'{stock_symbols[0]}'), ('closediff_week4_1', f'{stock_symbols[0]}'),
                 ('closediff_week3_2', f'{stock_symbols[0]}'), ('closediff_week4_2', f'{stock_symbols[0]}'), ('closediff_week4_3', f'{stock_symbols[0]}'),
                 ('total_closediff', f'{stock_symbols[0]}'), ('trend', f'{stock_symbols[0]}')]]

In [135]:
df_vcb = df_vcb[[('ema50', f'{stock_symbols[1]}'), ('ema21', f'{stock_symbols[1]}'), ('ema14', f'{stock_symbols[1]}'), ('ema5', f'{stock_symbols[1]}'), ('volume_total4week', f'{stock_symbols[1]}'),
                 ('rsi', f'{stock_symbols[1]}'), ('macd', f'{stock_symbols[1]}'), ('obv', f'{stock_symbols[1]}'), ('emv', f'{stock_symbols[1]}'),
                 ('mfi', f'{stock_symbols[1]}'), ('closediff_week2_1', f'{stock_symbols[1]}'), ('closediff_week3_1', f'{stock_symbols[1]}'), ('closediff_week4_1', f'{stock_symbols[1]}'),
                 ('closediff_week3_2', f'{stock_symbols[1]}'), ('closediff_week4_2', f'{stock_symbols[1]}'), ('closediff_week4_3', f'{stock_symbols[1]}'),
                 ('total_closediff', f'{stock_symbols[1]}'), ('trend', f'{stock_symbols[1]}')]]

In [136]:
df_vnm = df_vnm[[('ema50', f'{stock_symbols[2]}'), ('ema21', f'{stock_symbols[2]}'), ('ema14', f'{stock_symbols[2]}'), ('ema5', f'{stock_symbols[2]}'), ('volume_total4week', f'{stock_symbols[2]}'),
                 ('rsi', f'{stock_symbols[2]}'), ('macd', f'{stock_symbols[2]}'), ('obv', f'{stock_symbols[2]}'), ('emv', f'{stock_symbols[2]}'),
                 ('mfi', f'{stock_symbols[2]}'), ('closediff_week2_1', f'{stock_symbols[2]}'), ('closediff_week3_1', f'{stock_symbols[2]}'), ('closediff_week4_1', f'{stock_symbols[2]}'),
                 ('closediff_week3_2', f'{stock_symbols[2]}'), ('closediff_week4_2', f'{stock_symbols[2]}'), ('closediff_week4_3', f'{stock_symbols[2]}'),
                 ('total_closediff', f'{stock_symbols[2]}'), ('trend', f'{stock_symbols[2]}')]]

In [137]:
df_gas = df_gas[[('ema50', f'{stock_symbols[3]}'), ('ema21', f'{stock_symbols[3]}'), ('ema14', f'{stock_symbols[3]}'), ('ema5', f'{stock_symbols[3]}'), ('volume_total4week', f'{stock_symbols[3]}'),
                 ('rsi', f'{stock_symbols[3]}'), ('macd', f'{stock_symbols[3]}'), ('obv', f'{stock_symbols[3]}'), ('emv', f'{stock_symbols[3]}'),
                 ('mfi', f'{stock_symbols[3]}'), ('closediff_week2_1', f'{stock_symbols[3]}'), ('closediff_week3_1', f'{stock_symbols[3]}'), ('closediff_week4_1', f'{stock_symbols[3]}'),
                 ('closediff_week3_2', f'{stock_symbols[3]}'), ('closediff_week4_2', f'{stock_symbols[3]}'), ('closediff_week4_3', f'{stock_symbols[3]}'),
                 ('total_closediff', f'{stock_symbols[3]}'), ('trend', f'{stock_symbols[3]}')]]

In [138]:
df_hvn = df_hvn[[('ema50', f'{stock_symbols[4]}'), ('ema21', f'{stock_symbols[4]}'), ('ema14', f'{stock_symbols[4]}'), ('ema5', f'{stock_symbols[4]}'), ('volume_total4week', f'{stock_symbols[4]}'),
                 ('rsi', f'{stock_symbols[4]}'), ('macd', f'{stock_symbols[4]}'), ('obv', f'{stock_symbols[4]}'), ('emv', f'{stock_symbols[4]}'),
                 ('mfi', f'{stock_symbols[4]}'), ('closediff_week2_1', f'{stock_symbols[4]}'), ('closediff_week3_1', f'{stock_symbols[4]}'), ('closediff_week4_1', f'{stock_symbols[4]}'),
                 ('closediff_week3_2', f'{stock_symbols[4]}'), ('closediff_week4_2', f'{stock_symbols[4]}'), ('closediff_week4_3', f'{stock_symbols[4]}'),
                 ('total_closediff', f'{stock_symbols[4]}'), ('trend', f'{stock_symbols[4]}')]]

In [139]:
df_mwg = df_mwg[[('ema50', f'{stock_symbols[5]}'), ('ema21', f'{stock_symbols[5]}'), ('ema14', f'{stock_symbols[5]}'), ('ema5', f'{stock_symbols[5]}'), ('volume_total4week', f'{stock_symbols[5]}'),
                 ('rsi', f'{stock_symbols[5]}'), ('macd', f'{stock_symbols[5]}'), ('obv', f'{stock_symbols[5]}'), ('emv', f'{stock_symbols[5]}'),
                 ('mfi', f'{stock_symbols[5]}'), ('closediff_week2_1', f'{stock_symbols[5]}'), ('closediff_week3_1', f'{stock_symbols[5]}'), ('closediff_week4_1', f'{stock_symbols[5]}'),
                 ('closediff_week3_2', f'{stock_symbols[5]}'), ('closediff_week4_2', f'{stock_symbols[5]}'), ('closediff_week4_3', f'{stock_symbols[5]}'),
                 ('total_closediff', f'{stock_symbols[5]}'), ('trend', f'{stock_symbols[5]}')]]

In [140]:
df_fpt = df_fpt[[('ema50', f'{stock_symbols[6]}'), ('ema21', f'{stock_symbols[6]}'), ('ema14', f'{stock_symbols[6]}'), ('ema5', f'{stock_symbols[6]}'), ('volume_total4week', f'{stock_symbols[6]}'),
                 ('rsi', f'{stock_symbols[6]}'), ('macd', f'{stock_symbols[6]}'), ('obv', f'{stock_symbols[6]}'), ('emv', f'{stock_symbols[6]}'),
                 ('mfi', f'{stock_symbols[6]}'), ('closediff_week2_1', f'{stock_symbols[6]}'), ('closediff_week3_1', f'{stock_symbols[6]}'), ('closediff_week4_1', f'{stock_symbols[6]}'),
                 ('closediff_week3_2', f'{stock_symbols[6]}'), ('closediff_week4_2', f'{stock_symbols[6]}'), ('closediff_week4_3', f'{stock_symbols[6]}'),
                 ('total_closediff', f'{stock_symbols[6]}'), ('trend', f'{stock_symbols[6]}')]]

In [141]:
df_hpg = df_hpg[[('ema50', f'{stock_symbols[7]}'), ('ema21', f'{stock_symbols[7]}'), ('ema14', f'{stock_symbols[7]}'), ('ema5', f'{stock_symbols[7]}'), ('volume_total4week', f'{stock_symbols[7]}'),
                 ('rsi', f'{stock_symbols[7]}'), ('macd', f'{stock_symbols[7]}'), ('obv', f'{stock_symbols[7]}'), ('emv', f'{stock_symbols[7]}'),
                 ('mfi', f'{stock_symbols[7]}'), ('closediff_week2_1', f'{stock_symbols[7]}'), ('closediff_week3_1', f'{stock_symbols[7]}'), ('closediff_week4_1', f'{stock_symbols[7]}'),
                 ('closediff_week3_2', f'{stock_symbols[7]}'), ('closediff_week4_2', f'{stock_symbols[7]}'), ('closediff_week4_3', f'{stock_symbols[7]}'),
                 ('total_closediff', f'{stock_symbols[7]}'), ('trend', f'{stock_symbols[7]}')]]

In [142]:
df_msn = df_msn[[('ema50', f'{stock_symbols[8]}'), ('ema21', f'{stock_symbols[8]}'), ('ema14', f'{stock_symbols[8]}'), ('ema5', f'{stock_symbols[8]}'), ('volume_total4week', f'{stock_symbols[8]}'),
                 ('rsi', f'{stock_symbols[8]}'), ('macd', f'{stock_symbols[8]}'), ('obv', f'{stock_symbols[8]}'), ('emv', f'{stock_symbols[8]}'),
                 ('mfi', f'{stock_symbols[8]}'), ('closediff_week2_1', f'{stock_symbols[8]}'), ('closediff_week3_1', f'{stock_symbols[8]}'), ('closediff_week4_1', f'{stock_symbols[8]}'),
                 ('closediff_week3_2', f'{stock_symbols[8]}'), ('closediff_week4_2', f'{stock_symbols[8]}'), ('closediff_week4_3', f'{stock_symbols[8]}'),
                 ('total_closediff', f'{stock_symbols[8]}'), ('trend', f'{stock_symbols[8]}')]]

In [143]:
df_sab = df_sab[[('ema50', f'{stock_symbols[9]}'), ('ema21', f'{stock_symbols[9]}'), ('ema14', f'{stock_symbols[9]}'), ('ema5', f'{stock_symbols[9]}'), ('volume_total4week', f'{stock_symbols[9]}'),
                 ('rsi', f'{stock_symbols[9]}'), ('macd', f'{stock_symbols[9]}'), ('obv', f'{stock_symbols[9]}'), ('emv', f'{stock_symbols[9]}'),
                 ('mfi', f'{stock_symbols[9]}'), ('closediff_week2_1', f'{stock_symbols[9]}'), ('closediff_week3_1', f'{stock_symbols[9]}'), ('closediff_week4_1', f'{stock_symbols[9]}'),
                 ('closediff_week3_2', f'{stock_symbols[9]}'), ('closediff_week4_2', f'{stock_symbols[9]}'), ('closediff_week4_3', f'{stock_symbols[9]}'),
                 ('total_closediff', f'{stock_symbols[9]}'), ('trend', f'{stock_symbols[9]}')]]

In [144]:
stock_symbols

['VIC', 'VCB', 'VNM', 'GAS', 'HVN', 'MWG', 'FPT', 'HPG', 'MSN', 'SAB']

## Logistic Regression - Prediction & Valuation Model

#### VIC

In [165]:
X = df_vic.loc[:, df_vic.columns != ('trend', f'{stock_symbols[0]}')]
y = df_vic[('trend', f'{stock_symbols[0]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.2, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.2, random_state=42)

In [166]:
predictions = lr.predict(X_test_scaled)
predictions

array([1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1])

In [167]:
y_test.values

array([1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1])

In [168]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.77


#### VCB

In [169]:
X = df_vcb.loc[:, df_vcb.columns != ('trend', f'{stock_symbols[1]}')]
y = df_vcb[('trend', f'{stock_symbols[1]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.2, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.2, random_state=42)

In [170]:
predictions = lr.predict(X_test_scaled)
predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1])

In [171]:
y_test.values

array([1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1])

In [172]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.77


#### VNM

In [173]:
X = df_vnm.loc[:, df_vnm.columns != ('trend', f'{stock_symbols[2]}')]
y = df_vnm[('trend', f'{stock_symbols[2]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.1, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.1, random_state=42)

In [174]:
predictions = lr.predict(X_test_scaled)
predictions

array([1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1])

In [175]:
y_test.values

array([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1])

In [176]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.82


#### GAS

In [177]:
X = df_gas.loc[:, df_gas.columns != ('trend', f'{stock_symbols[3]}')]
y = df_gas[('trend', f'{stock_symbols[3]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.1, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.1, random_state=42)

In [178]:
predictions = lr.predict(X_test_scaled)
predictions

array([1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [179]:
y_test.values

array([1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0])

In [180]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.77


#### HVN

In [181]:
X = df_hvn.loc[:, df_hvn.columns != ('trend', f'{stock_symbols[4]}')]
y = df_hvn[('trend', f'{stock_symbols[4]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.2, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.2, random_state=42)

In [182]:
predictions = lr.predict(X_test_scaled)
predictions

array([0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])

In [183]:
y_test.values

array([0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0])

In [184]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.79


#### MWG

In [185]:
X = df_mwg.loc[:, df_mwg.columns != ('trend', f'{stock_symbols[5]}')]
y = df_mwg[('trend', f'{stock_symbols[5]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.1, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.1, random_state=42)

In [186]:
predictions = lr.predict(X_test_scaled)
predictions

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0])

In [187]:
y_test.values

array([1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0])

In [188]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.81


#### FPT

In [189]:
X = df_fpt.loc[:, df_fpt.columns != ('trend', f'{stock_symbols[6]}')]
y = df_fpt[('trend', f'{stock_symbols[6]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.2, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.2, random_state=42)

In [190]:
predictions = lr.predict(X_test_scaled)
predictions

array([1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0])

In [191]:
y_test.values

array([1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0])

In [192]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.73


#### HPG

In [197]:
X = df_hpg.loc[:, df_hpg.columns != ('trend', f'{stock_symbols[7]}')]
y = df_hpg[('trend', f'{stock_symbols[7]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.2, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.2, random_state=42)

In [198]:
predictions = lr.predict(X_test_scaled)
predictions

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1])

In [199]:
y_test.values

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0])

In [200]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.73


#### MSN

In [201]:
X = df_msn.loc[:, df_msn.columns != ('trend', f'{stock_symbols[8]}')]
y = df_msn[('trend', f'{stock_symbols[8]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.1, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.1, random_state=42)

In [202]:
predictions = lr.predict(X_test_scaled)
predictions

array([1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1])

In [203]:
y_test.values

array([0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1])

In [204]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.77


#### SAB

In [221]:
X = df_sab.loc[:, df_sab.columns != ('trend', f'{stock_symbols[9]}')]
y = df_sab[('trend', f'{stock_symbols[9]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21)

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)

lr = LogisticRegression(penalty='l2', C=0.05, random_state=42)

lr.fit(X_train_scaled, y_train.values)

LogisticRegression(C=0.05, random_state=42)

In [222]:
predictions = lr.predict(X_test_scaled)
predictions

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [223]:
y_test.values

array([0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1])

In [224]:
accuracy = accuracy_score(y_test.values, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.69


## Random Forest - Prediction & Valuation Model

#### VIC

In [225]:
X = df_vic[[('volume_total4week', f'{stock_symbols[0]}'), ('ema50', f'{stock_symbols[0]}'), ('ema21', f'{stock_symbols[0]}'),
            ('ema14', f'{stock_symbols[0]}'), ('ema5', f'{stock_symbols[0]}'), ('rsi', f'{stock_symbols[0]}'),
            ('macd', f'{stock_symbols[0]}'), ('obv', f'{stock_symbols[0]}'), ('emv', f'{stock_symbols[0]}'), ('mfi', f'{stock_symbols[0]}')]]
y = df_vic[('trend', f'{stock_symbols[0]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [226]:
y_pred = rf.predict(X_test.values)
y_pred

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0])

In [227]:
y_test.values

array([0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0])

In [228]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.73


## VCB

In [245]:
X = df_vcb[[('volume_total4week', f'{stock_symbols[1]}'), ('ema50', f'{stock_symbols[1]}'), ('ema21', f'{stock_symbols[1]}'),
            ('ema14', f'{stock_symbols[1]}'), ('ema5', f'{stock_symbols[1]}'), ('rsi', f'{stock_symbols[1]}'),
            ('macd', f'{stock_symbols[1]}'), ('obv', f'{stock_symbols[1]}'), ('emv', f'{stock_symbols[1]}'), ('mfi', f'{stock_symbols[1]}')]]
y = df_vcb[('trend', f'{stock_symbols[1]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [246]:
y_pred = rf.predict(X_test.values)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0])

In [247]:
y_test.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 0])

In [248]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.81


## VNM

In [253]:
X = df_vnm[[('volume_total4week', f'{stock_symbols[2]}'), ('ema50', f'{stock_symbols[2]}'), ('ema21', f'{stock_symbols[2]}'),
            ('ema14', f'{stock_symbols[2]}'), ('ema5', f'{stock_symbols[2]}'), ('rsi', f'{stock_symbols[2]}'),
            ('macd', f'{stock_symbols[2]}'), ('obv', f'{stock_symbols[2]}'), ('emv', f'{stock_symbols[2]}'), ('mfi', f'{stock_symbols[2]}')]]
y = df_vnm[('trend', f'{stock_symbols[2]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf = RandomForestClassifier(n_estimators=100, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(random_state=21)

In [254]:
y_pred = rf.predict(X_test.values)
y_pred

array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1])

In [255]:
y_test.values

array([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0])

In [256]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.72


## GAS

In [257]:
X = df_gas[[('volume_total4week', f'{stock_symbols[3]}'), ('ema50', f'{stock_symbols[3]}'), ('ema21', f'{stock_symbols[3]}'),
            ('ema14', f'{stock_symbols[3]}'), ('ema5', f'{stock_symbols[3]}'), ('rsi', f'{stock_symbols[3]}'),
            ('macd', f'{stock_symbols[3]}'), ('obv', f'{stock_symbols[3]}'), ('emv', f'{stock_symbols[3]}'), ('mfi', f'{stock_symbols[3]}')]]
y = df_gas[('trend', f'{stock_symbols[3]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [258]:
y_pred = rf.predict(X_test.values)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [259]:
y_test.values

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0])

In [260]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.68


## HVN

In [341]:
X = df_hvn[[('volume_total4week', f'{stock_symbols[4]}'), ('ema50', f'{stock_symbols[4]}'), ('ema21', f'{stock_symbols[4]}'),
            ('ema14', f'{stock_symbols[4]}'), ('ema5', f'{stock_symbols[4]}'), ('rsi', f'{stock_symbols[4]}'),
            ('macd', f'{stock_symbols[4]}'), ('obv', f'{stock_symbols[4]}'), ('emv', f'{stock_symbols[4]}'), ('mfi', f'{stock_symbols[4]}')]]
y = df_hvn[('trend', f'{stock_symbols[4]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [342]:
y_pred = rf.predict(X_test.values)
y_pred

array([1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [343]:
y_test.values

array([0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0])

In [344]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.59


## MWG

In [352]:
X = df_mwg[[('volume_total4week', f'{stock_symbols[5]}'), ('ema50', f'{stock_symbols[5]}'), ('ema21', f'{stock_symbols[5]}'),
            ('ema14', f'{stock_symbols[5]}'), ('ema5', f'{stock_symbols[5]}'), ('rsi', f'{stock_symbols[5]}'),
            ('macd', f'{stock_symbols[5]}'), ('obv', f'{stock_symbols[5]}'), ('emv', f'{stock_symbols[5]}'), ('mfi', f'{stock_symbols[5]}')]]
y = df_mwg[('trend', f'{stock_symbols[5]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [353]:
y_pred = rf.predict(X_test.values)
y_pred

array([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [354]:
y_test.values

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0])

In [355]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.82


## FPT

In [356]:
X = df_fpt[[('volume_total4week', f'{stock_symbols[6]}'), ('ema50', f'{stock_symbols[6]}'), ('ema21', f'{stock_symbols[6]}'),
            ('ema14', f'{stock_symbols[6]}'), ('ema5', f'{stock_symbols[6]}'), ('rsi', f'{stock_symbols[6]}'),
            ('macd', f'{stock_symbols[6]}'), ('obv', f'{stock_symbols[6]}'), ('emv', f'{stock_symbols[6]}'), ('mfi', f'{stock_symbols[6]}')]]
y = df_fpt[('trend', f'{stock_symbols[6]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [357]:
y_pred = rf.predict(X_test.values)
y_pred

array([1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1])

In [358]:
y_test.values

array([1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1])

In [359]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.73


## HPG

In [368]:
X = df_hpg[[('volume_total4week', f'{stock_symbols[7]}'), ('ema50', f'{stock_symbols[7]}'), ('ema21', f'{stock_symbols[7]}'),
            ('ema14', f'{stock_symbols[7]}'), ('ema5', f'{stock_symbols[7]}'), ('rsi', f'{stock_symbols[7]}'),
            ('macd', f'{stock_symbols[7]}'), ('obv', f'{stock_symbols[7]}'), ('emv', f'{stock_symbols[7]}'), ('mfi', f'{stock_symbols[7]}')]]
y = df_hpg[('trend', f'{stock_symbols[7]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [369]:
y_pred = rf.predict(X_test.values)
y_pred

array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0])

In [370]:
y_test.values

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1])

In [371]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.61


## MSN

In [378]:
X = df_msn[[('volume_total4week', f'{stock_symbols[8]}'), ('ema50', f'{stock_symbols[8]}'), ('ema21', f'{stock_symbols[8]}'),
            ('ema14', f'{stock_symbols[8]}'), ('ema5', f'{stock_symbols[8]}'), ('rsi', f'{stock_symbols[8]}'),
            ('macd', f'{stock_symbols[8]}'), ('obv', f'{stock_symbols[8]}'), ('emv', f'{stock_symbols[8]}'), ('mfi', f'{stock_symbols[8]}')]]
y = df_msn[('trend', f'{stock_symbols[8]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [379]:
y_pred = rf.predict(X_test.values)
y_pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1])

In [380]:
y_test.values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0])

In [381]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.74


## SAB

In [387]:
X = df_sab[[('volume_total4week', f'{stock_symbols[9]}'), ('ema50', f'{stock_symbols[9]}'), ('ema21', f'{stock_symbols[9]}'),
            ('ema14', f'{stock_symbols[9]}'), ('ema5', f'{stock_symbols[9]}'), ('rsi', f'{stock_symbols[9]}'),
            ('macd', f'{stock_symbols[9]}'), ('obv', f'{stock_symbols[9]}'), ('emv', f'{stock_symbols[9]}'), ('mfi', f'{stock_symbols[9]}')]]
y = df_sab[('trend', f'{stock_symbols[9]}')]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf = RandomForestClassifier(n_estimators=110, random_state=21)
rf.fit(X_train.values, y_train.values)

RandomForestClassifier(n_estimators=110, random_state=21)

In [388]:
y_pred = rf.predict(X_test.values)
y_pred

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1])

In [389]:
y_test.values

array([1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0])

In [390]:
accuracy = accuracy_score(y_test.values, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.69


## THE END